In [1]:
#   This is the Python implementation/translation of the Matlab code on the Handbook Chapter
#      Jacob Schettewi contributed to the code. 
#
import numpy as np
import pandas as pd
import itertools
import xlwings as xw
import scipy.special as sps
from scipy.stats import zscore
from sklearn.decomposition import PCA
from scipy.special import beta as beta_func, betainc
from scipy.stats import norm
from scipy.stats import t as tdist

In [2]:
def beta_inv(p, a, b):
    """
    PURPOSE: inverse of the cdf (quantile) of the beta(a,b) distribution
    --------------------------------------------------------------
    USAGE: x = beta_inv(p,a,b)
    where:   p = vector of probabilities
             a = beta distribution parameter, a = scalar
             b = beta distribution parameter  b = scalar
    NOTE: mean [beta(a,b)] = a/(a+b), variance = ab/((a+b)*(a+b)*(a+b+1))
    --------------------------------------------------------------
    RETURNS: x at each element of p for the beta(a,b) distribution
    """
    # check number of arguments
    if len([arg for arg in (p, a, b) if arg is not None]) != 3:
        raise ValueError("Wrong # of arguments to beta_inv")

    # check parameter values
    if (a <= 0).any() or (b <= 0).any():
        raise ValueError("beta_inv parameter a or b is nonpositive")
    if ((2 * p - 1).abs() > 1).any() or (p < 0).any() or (p > 1).any():
        raise ValueError("beta_inv: A probability should be 0<=p<=1")

    x = a / (a + b)
    dx = np.ones_like(x)
    while np.max(np.abs(dx)) > 256 * np.finfo(float).eps * np.max(np.abs(x)):
        dx = (betainc(a, b, x) - p) / beta_func(a, b) / x ** (a - 1) / (1 - x) ** (b - 1)
        x = x - dx
        x = x + (dx - x) / 2 * (x < 0)

    return x

In [3]:
def beta_pdf(x, a, b):
    """
    PURPOSE: pdf of the beta(a,b) distribution
    --------------------------------------------------------------
    USAGE: pdf = beta_pdf(x,a,b)
    where:   x = vector of components
             a = beta distribution parameter, a = scalar
             b = beta distribution parameter  b = scalar
    NOTE: mean[(beta(a,b)] = a/(a+b), variance = ab/((a+b)*(a+b)*(a+b+1))
    --------------------------------------------------------------
    RETURNS: pdf at each element of x of the beta(a,b) distribution
    """
    # check number of arguments
    if len([arg for arg in (x, a, b) if arg is not None]) != 3:
        raise ValueError("Wrong # of arguments to beta_pdf")

    # check parameter values
    if (a <= 0) or (b <= 0):
        raise ValueError("Parameter a or b is nonpositive")

    # find indices of x outside [0,1] and set pdf to zero at those indices
    I = np.where((x < 0) | (x > 1))[0]
    pdf = np.power(x, a - 1) * np.power(1 - x, b - 1) / beta_func(a, b)
    pdf[I] = 0

    return pdf

In [4]:
def ols(y, x):
    """
    PURPOSE: least-squares regression 
    ---------------------------------------------------
    USAGE: results = ols(y,x)
    where: y = dependent variable vector    (nobs x 1)
           x = independent variables matrix (nobs x nvar)
    ---------------------------------------------------
    RETURNS: a dictionary
           results['meth']  = 'ols'
           results['beta']  = bhat     (nvar x 1)
           results['tstat'] = t-stats  (nvar x 1)
           results['bstd']  = std deviations for bhat (nvar x 1)
           results['yhat']  = yhat     (nobs x 1)
           results['resid'] = residuals (nobs x 1)
           results['sige']  = e'*e/(n-k)   scalar
           results['rsqr']  = rsquared     scalar
           results['rbar']  = rbar-squared scalar
           results['dw']    = Durbin-Watson Statistic
           results['nobs']  = nobs
           results['nvar']  = nvars
           results['y']     = y data vector (nobs x 1)
           results['bint']  = (nvar x2 ) vector with 95% confidence intervals on beta
    """
    if len(y) != x.shape[0]:
        raise ValueError('x and y must have same # obs in ols')

    nobs, nvar = x.shape
    results = {}

    results['meth'] = 'ols'
    results['y'] = y
    results['nobs'] = nobs
    results['nvar'] = nvar

    if nobs < 10000:
        q, r = np.linalg.qr(x)
        xpxi = np.linalg.inv(r.T @ r)  # equivalent to (r'*r)\eye(nvar)
    else:  # use Cholesky for very large problems
        xpxi = np.linalg.inv(x.T @ x)

    results['beta'] = xpxi @ (x.T @ y)
    results['yhat'] = x @ results['beta']
    results['resid'] = y - results['yhat']
    sigu = results['resid'].T @ results['resid']
    results['sige'] = sigu / (nobs - nvar)
    tmp = results['sige'] * np.diag(xpxi)
    sigb = np.sqrt(tmp)
    results['bstd'] = sigb
    tcrit = tdist.ppf(0.025, nobs - nvar)
    results['bint'] = np.column_stack((results['beta'] - tcrit * sigb, results['beta'] + tcrit * sigb))
    results['tstat'] = results['beta'] / (np.sqrt(tmp))
    ym = y - np.mean(y)
    rsqr1 = sigu
    rsqr2 = ym.T @ ym
    results['rsqr'] = 1.0 - rsqr1/rsqr2  # r-squared
    rsqr1 = rsqr1/(nobs-nvar)
    rsqr2 = rsqr2/(nobs-1.0)
    if rsqr2 != 0:
        results['rbar'] = 1 - (rsqr1/rsqr2)  # rbar-squared
    else:
        results['rbar'] = results.rsqr
    ediff = results['resid'][1:nobs-1] - results['resid'][0:nobs-2]
    results['dw'] = (ediff.T @ ediff)/sigu  # durbin-watson

    return results

In [5]:
def tdis_inv(p, a):
    """
    PURPOSE: returns the inverse (quantile) at x of the t(n) distribution
    ---------------------------------------------------
    USAGE: x = tdis_inv(p,n)
    where: p = a vector of probabilities 
           n = a scalar dof parameter
    ---------------------------------------------------
    RETURNS:
           a vector of tinv at each element of x of the t(n) distribution      
    """
    s = p < 0.5
    p = p + (1 - 2 * p) * s
    p = 1 - (2 * (1 - p))
    x = sps.betaincinv(1/2, a/2, p)
    x = x * a / ((1 - x))
    x = (1 - 2 * s) * np.sqrt(x)
    return x

In [6]:
def Perform_asset_allocation(actual, risk_free, forecast, volatility_forecast, gamma_MV):
    """
    Performs asset allocation evaluation for allocation between risky
    asset and risk-free bill. The risky equity is restricted to lie
    between zero and 1.5.
    
    Input:
    
    actual              = T-vector of actual excess returns
    risk_free           = T-vector of risk-free rates
    forecast            = T-vector of excess return forecasts
    volatility_forecast = T-vector of volatility forecasts
    gamma_MV            = risk aversion parameter
    
    Output:
    
    avg_utility  = average utility
    weight_risky = T-vector of risky portfolio weights
    """
    T = actual.shape[0]
    weight_risky = np.zeros(T)
    for iter in range(T):
        weight_risky[iter] = (1 / gamma_MV) * (forecast[iter] / volatility_forecast[iter])
        if weight_risky[iter] < 0:
            weight_risky[iter] = 0
        elif weight_risky[iter] > 1.5:
            weight_risky[iter] = 1.5
    return_portfolio = risk_free + weight_risky * actual
    avg_utility = np.mean(return_portfolio) - 0.5 * gamma_MV * np.std(return_portfolio, ddof=1)**2
    return avg_utility, weight_risky


In [7]:
def nwest(y, x, nlag):
    """
    PURPOSE: computes Newey-West adjusted heteroscedastic-serial
             consistent Least-squares Regression
    ---------------------------------------------------
    USAGE: results = nwest(y,x,nlag)
    where: y = dependent variable vector (nobs x 1)
           x = independent variables matrix (nobs x nvar)
        nlag = lag length to use
    ---------------------------------------------------
    RETURNS: a dictionary
           results['meth']  = 'newlyw'
           results['beta']  = bhat
           results['tstat'] = t-stats
           results['yhat']  = yhat
           results['resid'] = residuals
           results['sige']  = e'*e/(n-k)
           results['rsqr']  = rsquared
           results['rbar']  = rbar-squared
           results['dw']    = Durbin-Watson Statistic
           results['nobs']  = nobs
           results['nvar']  = nvars
           results['y']     = y data vector
    """
    # check if number of arguments is correct
    if len([y, x, nlag]) != 3:
        raise ValueError('Wrong number of arguments to nwest')
        
    nobs, nvar = x.shape
    
    results = {'meth': 'nwest',
               'y': y,
               'nobs': nobs,
               'nvar': nvar}
    
    xpxi = np.linalg.inv(x.T @ x)
    results['beta'] = xpxi @ x.T @ y
    results['yhat'] = x @ results['beta']
    results['resid'] = y - results['yhat']
    sigu = results['resid'].T @ results['resid']
    results['sige'] = sigu / (nobs - nvar)
    
    # perform Newey-West correction
    emat = np.zeros((nvar * nobs, nvar))
    for i in range(nvar):
        emat[i*nobs:(i+1)*nobs, i] = results['resid']
        
    hhat = emat * x
    G = np.zeros((nvar, nvar))
    w = np.zeros((2*nlag+1, 1))
    
    for a in range(nlag+1):
        ga = np.zeros((nvar, nvar))
        w[nlag+a, 0] = (nlag+1-a)/(nlag+1)
        za = hhat[:, a:nobs].T @ hhat[:, :nobs]
        if a == 0:
            ga += za
        else:
            ga += za + za.T
        G += w[nlag+a, 0] * ga
        
    V = xpxi @ G @ xpxi
    nwerr = np.sqrt(np.diag(V))
    results['tstat'] = results['beta'] / nwerr
    
    ym = y - np.mean(y)
    rsqr1 = sigu
    rsqr2 = ym.T @ ym
    results['rsqr'] = 1 - rsqr1/rsqr2
    rsqr1 = rsqr1 / (nobs - nvar)
    rsqr2 = rsqr2 / (nobs - 1)
    results['rbar'] = 1 - rsqr1/rsqr2
    ediff = results['resid'][1:] - results['resid'][:-1]
    results['dw'] = np.diag((ediff.reshape(-1,1).T @ ediff.reshape(-1,1)) / sigu)
    
    return results


# Forecasts Monthly Log

## Loading data and defining variables, 1926:12-2010:12

In [8]:
data_m = pd.read_excel('Returns_handbook_data.xls', sheet_name='Monthly')

In [9]:
data_m

Date (yyyymm)  S&P 500 index  12-month moving sum of S&P 500 dividends  \
0          187101.0           4.44                                  0.260000   
1          187102.0           4.50                                  0.260000   
2          187103.0           4.61                                  0.260000   
3          187104.0           4.74                                  0.260000   
4          187105.0           4.86                                  0.260000   
...             ...            ...                                       ...   
1677       201010.0        1183.26                                 22.480667   
1678       201011.0        1180.55                                 22.605333   
1679       201012.0        1257.64                                 22.730000   
1680            NaN            NaN                                       NaN   
1681            NaN            NaN                                       NaN   

      12-month moving sum of S&P 500 earnings  \
0                                        0.40   
1                                        0.40   
2                                        0.40   
3                                        0.40   
4                                        0.40   
...                                       ...   
1677                                    73.69   
1678                                    75.52   
1679                                    77.35   
1680                                      NaN   
1681                                      NaN   

      DJIA book-to-market value ratio  \
0                                 NaN   
1                                 NaN   
2                                 NaN   
3                                 NaN   
4                                 NaN   
...                               ...   
1677                         0.372887   
1678                         0.376695   
1679                         0.358100   
1680                              NaN   
1681                              NaN   

      3-month Treasury bill yield (secondary market)  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   
...                                              ...   
1677                                          0.0013   
1678                                          0.0014   
1679                                          0.0014   
1680                                             NaN   
1681                                             NaN   

      Moodys AAA-rated corporate bond yield  \
0                                       NaN   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   
...                                     ...   
1677                                 0.0468   
1678                                 0.0487   
1679                                 0.0502   
1680                                    NaN   
1681                                    NaN   

      Moodys BAA-rated corporate bond yield  Long-term government bond yield  \
0                                       NaN                              NaN   
1                                       NaN                              NaN   
2                                       NaN                              NaN   
3                                       NaN                              NaN   
4                                       NaN                              NaN   
...                                     ...                              ...   
1677                                 0.0572                         0.036716   
1678                                 0.0592                         0.038026   
1679 

In [10]:
# Equity premium

market_return = data_m.iloc[671:1680, 15].values  # S&P 500 VW return
r_f_lag = data_m.iloc[670:1679, 10].values # risk-free rate, lagged (1926:11-2010:11)
equity_premium = np.log(1 + market_return) - np.log(1 + r_f_lag) # log excess return

In [11]:
# Predictors

D12 = data_m.iloc[671:1680, 2].values
SP500 = data_m.iloc[671:1680, 1].values  # S&P 500 index
DP = np.log(D12) - np.log(SP500)  # log dividend-price ratio
SP500_lag = data_m.iloc[670:1679, 1].values  # S&P 500 index, lagged (1926:11-2010:11)
DY = np.log(D12) - np.log(SP500_lag)  # log dividend yield
E12 = data_m.iloc[671:1680, 3].values  # earnings
EP = np.log(E12) - np.log(SP500)  # log earnings-price ratio
DE = np.log(D12) - np.log(E12)  # log dividend-payout ratio
SVAR = data_m.iloc[671:1680, 14].values  # volatility
BM = data_m.iloc[671:1680, 4].values  # book-to-market ratio
NTIS = data_m.iloc[671:1680, 9].values  # net equity issuing activity
TBL = data_m.iloc[671:1680, 5].values  # T-bill rate
LTY = data_m.iloc[671:1680, 8].values  # long-term government bond yield
LTR = data_m.iloc[671:1680, 12].values  # long-term government bond return
TMS = LTY - TBL  # term spread
AAA = data_m.iloc[671:1680, 6].values  # AAA-rated corporate bond yield
BAA = data_m.iloc[671:1680, 7].values  # BAA-rated corporate bond yield
DFY = BAA - AAA  # default yield spread
CORPR = data_m.iloc[671:1680, 13].values  # long-term corporate bond return
DFR = CORPR - LTR  # default return spread
INFL_lag = data_m.iloc[670:1679, 11].values  # inflation, lagged (1926:11-2010:11)
ECON = np.column_stack((DP, DY, EP, DE, SVAR, BM, NTIS, TBL, LTY, LTR, TMS, DFY, DFR, INFL_lag))
ECON_sink = np.column_stack((DP, DY, EP, SVAR, BM, NTIS, TBL, LTY, LTR, DFY, DFR, INFL_lag))

In [12]:
# Sum-of-the-parts variables

E12_lag = data_m.iloc[670:1679, 3].values  # earnings, lagged (1926:11-2010:11)
E_growth = np.log((1 / 12) * E12) - np.log((1 / 12) * E12_lag)
DP_SOP = np.log(1 + (1 / 12) * np.divide(D12, SP500)) # log (1+D/P)
r_f = data_m.iloc[671:1680, 10].values # risk-free rate
r_f = np.log(1 + r_f) # log risk-free rate

## Estimating full-sample parameters for Campbell-Thompson restrictions

In [13]:
beta_full = np.zeros(np.shape(ECON)[1])
T = np.shape(equity_premium)[0]
LHS = equity_premium[1:T]

In [14]:
for i in range(ECON.shape[1]):
    RHS_i = np.hstack((ECON[0:T-1, i].reshape(-1, 1), np.ones((T-1, 1))))
    results_i = ols(LHS, RHS_i)
    beta_full[i] = results_i['beta'][0]
    print(i)
beta_full[4] = 1 # restricting SVAR slope coefficient to be positive

# NB: following Campbell and Thompson (2008) by using full-sample
# estimates as theoretical 'priors' (although we make sure that
# SVAR slope coefficient is positive)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


## Preliminaries

In [15]:
Y = equity_premium
T = np.shape(Y)[0]
N = np.shape(ECON)[1]
R = (1946 - 1926) * 12 + 1 # in-sample period, 1926:12-1946:12
P_0 = (1956 - 1946) * 12 # holdout out-of-sample period, 1947:01-1956:12
P = T - (R + P_0) # forecast evaluation period, 1957:01-2010:12
theta = 0.75 # discound factor for DMSFE pooled forecast
r = 1 # number of principal components
MA_SOP = 20 * 12 # window size for SOP forecast

## Computing forecasts, 1947:01-2010:12 (include holdout OOS period)

In [16]:
FC_HA = np.zeros((P_0 + P, 1))
FC_ECON = np.zeros((P_0 + P, N))
beta_ECON = np.zeros((P_0 + P, N, 2))
FC_ECON_CT = np.zeros((P_0 + P, N))
omega_DMSFE = np.zeros((P_0 + P, N))
FC_OTHER = np.zeros((P_0 + P, 6))
FC_OTHER_CT = np.zeros((P_0 + P, 6))

In [17]:
for t in range(P_0 + P):

    FC_HA[t] = np.mean(Y[0:R+t])
    
    # Individual predictive regression forecasts

    X_t = ECON[0:R+t-1,:]
    Y_t = Y[1:R+t]
    for i in range(N):
        results_t_i = ols(Y_t, np.hstack((X_t[:,i:i+1], np.ones((R+t-1,1)))))
        FC_ECON[t,i] = np.dot(np.array([[ECON[R+(t-1),i], 1]]), results_t_i['beta']) # scaler output
        beta_ECON[t,i,0] = results_t_i['beta'][0]
        beta_ECON[t,i,1] = results_t_i['bstd'][0]
        if beta_full[i] > 0:
            if results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        elif beta_full[i] < 0:
            if results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        if FC_ECON_CT[t,i] < 0:
            FC_ECON_CT[t,i] = 0
    if t >= P_0:

        # Kitchen sink forecast

        X_t_sink = ECON_sink[0:R+t-1,:]
        results_t_sink = ols(Y_t, np.hstack((X_t_sink, np.ones((R+t-1,1)))))
        FC_OTHER[t,0] = np.dot(np.append(ECON_sink[R+(t-1),:], 1), results_t_sink['beta'])
        if FC_OTHER[t,0] < 0:
            FC_OTHER_CT[t,0] = 0
        else:
            FC_OTHER_CT[t,0] = FC_OTHER[t,0]
        
        # SIC forecast
        
        j_max = 3  # consider models with up to 3 predictors to save time
        SIC_t = np.empty((0, 2))
        for j in range(1, j_max + 1):
            select_j = list(itertools.combinations(range(X_t_sink.shape[1]), j))
            for k in range(len(select_j)):
                X_t_j_k = np.hstack((X_t_sink[0:R+t-1,select_j[k]], np.ones((R+t-1,1))))
                results_t_j_k = ols(Y_t, X_t_j_k)
                SIC_t_j_k = np.log(np.dot(results_t_j_k['resid'].T, results_t_j_k['resid']) / np.shape(Y_t)[0]) + \
                    np.log(np.shape(Y_t)[0]) * np.shape(X_t_j_k)[1] / np.shape(Y_t)[0]
                FC_t_j_k = np.hstack((ECON_sink[R+(t-1),select_j[k]], 1)).dot(results_t_j_k['beta'])
                SIC_t = np.vstack((SIC_t, [SIC_t_j_k, FC_t_j_k]))
        SIC_t_min_index = np.argmin(SIC_t[:,0])
        SIC_t_min = SIC_t[SIC_t_min_index,0]
        FC_OTHER[t,1] = SIC_t[SIC_t_min_index,1];
        if FC_OTHER[t,1] < 0:
            FC_OTHER_CT[t,1] = 0
        else:
            FC_OTHER_CT[t,1] = FC_OTHER[t,1]

        # Pooled forecast: simple average

        FC_OTHER[t,2] = np.mean(FC_ECON[t,:])
        if FC_OTHER[t,2] < 0:
            FC_OTHER_CT[t,2] = 0
        else:
            FC_OTHER_CT[t,2] = FC_OTHER[t,2]
            

        # Pooled forecast: DMSFE

        powers_t = np.arange(t-1, -1, -1).reshape(-1, 1)
        m = np.sum((np.kron(np.ones((1, N)), (theta*np.ones((t, 1))))**powers_t) * 
                   (((np.kron(Y[R:R+t], np.ones((1, N))).reshape(t, N) - FC_ECON[0:t, :])**2)), axis=0)
        omega = m**(-1) / np.sum(m**(-1))
        omega_DMSFE[t, :] = omega
        FC_OTHER[t, 3] = FC_ECON[t, :].dot(omega)
        if FC_OTHER[t,3] < 0:
            FC_OTHER_CT[t,3] = 0
        else:
            FC_OTHER_CT[t,3] = FC_OTHER[t,3]

        # Diffusion index forecast

        X_t_DI_standardize = zscore(ECON[0:R+t, :])
        pca = PCA()
        pca.fit(X_t_DI_standardize)
        Lambda_t = pca.components_.T
        F_t = pca.transform(X_t_DI_standardize)
        results_t_DI = ols(Y_t, np.hstack((F_t[0:-1, 0:r], np.ones((R+t-1,1)))))
        FC_OTHER[t,4] = np.dot(np.append(F_t[-1,0:r], 1), results_t_DI['beta'])
        if FC_OTHER[t,4] < 0:
            FC_OTHER_CT[t,4] = 0
        else:
            FC_OTHER_CT[t,4] = FC_OTHER[t,4]

        # Sum-of-the-parts forecast

        FC_OTHER[t, 5] = np.mean(E_growth[R+(t-1)-MA_SOP+1:R+t]) + DP_SOP[R+(t-1)] - r_f[R+(t-1)]

        if FC_OTHER[t,5] < 0:
            FC_OTHER_CT[t,5] = 0
        else:
            FC_OTHER_CT[t,5] = FC_OTHER[t,5]
        
    print(t, FC_HA[t], FC_OTHER[t, :])

0 [0.00357551] [0. 0. 0. 0. 0. 0.]
1 [0.00364943] [0. 0. 0. 0. 0. 0.]
2 [0.00359878] [0. 0. 0. 0. 0. 0.]
3 [0.00352056] [0. 0. 0. 0. 0. 0.]
4 [0.00334019] [0. 0. 0. 0. 0. 0.]
5 [0.00332129] [0. 0. 0. 0. 0. 0.]
6 [0.0035194] [0. 0. 0. 0. 0. 0.]
7 [0.00366043] [0. 0. 0. 0. 0. 0.]
8 [0.00354975] [0. 0. 0. 0. 0. 0.]
9 [0.00349324] [0. 0. 0. 0. 0. 0.]
10 [0.00355782] [0. 0. 0. 0. 0. 0.]
11 [0.00347028] [0. 0. 0. 0. 0. 0.]
12 [0.00357338] [0. 0. 0. 0. 0. 0.]
13 [0.00340473] [0. 0. 0. 0. 0. 0.]
14 [0.00322718] [0. 0. 0. 0. 0. 0.]
15 [0.00351716] [0. 0. 0. 0. 0. 0.]
16 [0.00361834] [0. 0. 0. 0. 0. 0.]
17 [0.00391378] [0. 0. 0. 0. 0. 0.]
18 [0.00392494] [0. 0. 0. 0. 0. 0.]
19 [0.00370024] [0. 0. 0. 0. 0. 0.]
20 [0.00374186] [0. 0. 0. 0. 0. 0.]
21 [0.00361981] [0. 0. 0. 0. 0. 0.]
22 [0.00385972] [0. 0. 0. 0. 0. 0.]
23 [0.00344942] [0. 0. 0. 0. 0. 0.]
24 [0.00356611] [0. 0. 0. 0. 0. 0.]
25 [0.00355912] [0. 0. 0. 0. 0. 0.]
26 [0.00342172] [0. 0. 0. 0. 0. 0.]
27 [0.00353001] [0. 0. 0. 0. 0. 0.]
28 

164 [0.00648496] [0.00320167 0.00823023 0.00455346 0.00455229 0.00173492 0.00559744]
165 [0.00631542] [-0.01132886  0.00961223  0.00385393  0.00363664  0.00170654  0.00558307]
166 [0.00629191] [-0.00289312  0.0094448   0.00428241  0.00401572  0.0019171   0.0057789 ]
167 [0.00638185] [-0.00069802  0.00934669  0.00421831  0.00418768  0.0017371   0.00564233]
168 [0.0064778] [-0.00654467  0.00941549  0.00366736  0.00379351  0.00215665  0.00565261]
169 [0.00660928] [0.00751084 0.00858467 0.00449588 0.00476406 0.00167874 0.00540042]
170 [0.00666719] [-0.00345365  0.00793872  0.00376841  0.00415475  0.00168433  0.00508599]
171 [0.00671624] [0.00565039 0.01108037 0.00405171 0.00453677 0.00162137 0.00495004]
172 [0.00670619] [-0.00346947  0.01064301  0.00352712  0.00399558  0.00183509  0.00495755]
173 [0.0067414] [0.00418838 0.00986379 0.00409003 0.00470988 0.00157606 0.00484518]
174 [0.00665541] [-0.00362946  0.00902617  0.00388281  0.00402523  0.00141547  0.00481806]
175 [0.00671456] [-0.0006

259 [0.00626748] [-0.0031816   0.01147804  0.00406946  0.00402449  0.00034369  0.00258066]
260 [0.00627865] [0.00710792 0.01104032 0.00444323 0.00447571 0.00044953 0.0026193 ]
261 [0.0063346] [0.01220983 0.01009911 0.00445431 0.00468482 0.00047783 0.00233179]
262 [0.00633042] [0.00776329 0.00948367 0.00432458 0.00455932 0.00023943 0.00207161]
263 [0.0064126] [0.01365535 0.009328   0.00446201 0.00489805 0.00028226 0.00177105]
264 [0.00630839] [ 0.00555433  0.00975335  0.00414331  0.00410031 -0.00060405  0.00134912]
265 [0.00627116] [ 0.00342222  0.00930975  0.00395796  0.00381062 -0.00077551  0.00118843]
266 [0.00616343] [-0.00884768  0.00979617  0.0033441   0.00303827 -0.00067058  0.00129794]
267 [0.00620871] [ 0.00486305  0.01284276  0.00363085  0.00351229 -0.00025336  0.00126338]
268 [0.00623092] [-0.00371833  0.01244247  0.00323832  0.00321961 -0.00026384  0.00113652]
269 [0.00621628] [ 0.01022114  0.01273452  0.00392543  0.00383581 -0.00074781  0.00120128]
270 [0.00608492] [-0.0046

351 [0.0050714] [0.00729269 0.00934034 0.00512695 0.00541397 0.00264617 0.00238418]
352 [0.00503956] [0.00011996 0.00936495 0.00503041 0.00523764 0.00264461 0.00268018]
353 [0.00501055] [-0.00442376  0.00995927  0.00481427  0.00493478  0.00207762  0.00258742]
354 [0.00506716] [0.00488846 0.00921098 0.00495828 0.00525592 0.00179226 0.00243406]
355 [0.00503899] [0.00204549 0.00969692 0.00496384 0.00514542 0.0018449  0.002731  ]
356 [0.00502594] [0.00374269 0.00999662 0.00509783 0.00521306 0.00200056 0.00294497]
357 [0.00505073] [0.00623285 0.00952681 0.00502811 0.0052885  0.00184066 0.00304985]
358 [0.00500116] [0.00064011 0.01021138 0.00503089 0.00495038 0.00213953 0.00337772]
359 [0.00498427] [0.00724797 0.01073045 0.0053799  0.00518946 0.00257801 0.00366943]
360 [0.00505656] [0.01510427 0.00908482 0.0055142  0.00575049 0.0028882  0.00396247]
361 [0.00495755] [-0.00357843  0.01045427  0.00481435  0.00469457  0.00254958  0.00397074]
362 [0.00491681] [-0.00351845  0.01095207  0.00512724 

442 [0.00462551] [-0.02699464  0.00495548  0.00401088  0.00403274  0.00300071  0.00150392]
443 [0.00464271] [-0.02133729  0.00416974  0.00415836  0.00421854  0.00305571  0.00141181]
444 [0.0046169] [-0.0240069   0.0044117   0.00397624  0.00394555  0.00280328  0.0013119 ]
445 [0.00459358] [-0.01594125  0.00500051  0.00432922  0.00413415  0.00346569  0.00150816]
446 [0.00452574] [-0.02544214  0.00608653  0.00399658  0.00366547  0.00360463  0.00157557]
447 [0.00453311] [-0.02207852  0.00606474  0.00402823  0.00377208  0.00348399  0.00124529]
448 [0.00452487] [-0.0184563   0.00594678  0.00425262  0.00394151  0.00367819  0.001142  ]
449 [0.00442754] [-0.03322201 -0.01176185  0.00395409  0.0034749   0.00406052  0.00132063]
450 [0.00444214] [-0.01643073 -0.01191533  0.00506258  0.00472548  0.00512603  0.00127524]
451 [0.00440386] [-0.01300192 -0.00989336  0.00528825  0.00474936  0.00524228  0.00113381]
452 [0.00453914] [-0.00276305  0.00501992  0.00494415  0.00512348  0.00440008  0.00045424]


531 [0.00483055] [-0.0032862   0.00907147  0.00370187  0.00384758  0.0013257   0.00356119]
532 [0.00482088] [-0.00638834 -0.00113516  0.00362924  0.0037563   0.00143561  0.00364959]
533 [0.00486289] [-0.0061966   0.00827699  0.00349886  0.0037204   0.00125899  0.00357863]
534 [0.00479086] [-0.01639999 -0.00124863  0.0032172   0.00318143  0.00109247  0.00348627]
535 [0.00483732] [-0.00907191  0.00639582  0.00330989  0.00338905  0.00111593  0.00322648]
536 [0.00485551] [-0.00961035  0.00582988  0.00327288  0.0034166   0.0012939   0.00324556]
537 [0.00482171] [-0.01422085 -0.00157053  0.00308165  0.00308219  0.00104559  0.00324596]
538 [0.00482711] [-0.01743342  0.0039251   0.00280371  0.00284708  0.00113983  0.00318509]
539 [0.00476298] [-0.01873587 -0.00117606  0.00292203  0.00273584  0.00187144  0.00343583]
540 [0.00489036] [-0.0063016   0.00255098  0.0030605   0.00319098  0.00175802  0.00334132]
541 [0.00485609] [-0.01839832  0.00248505  0.00270779  0.00278534  0.00138359  0.00361546]

620 [0.0052564] [-0.01037868  0.0036991   0.00245346  0.00233231 -0.00132246  0.00236179]
621 [0.00531664] [ 0.01411607  0.00480502  0.00321557  0.00317559 -0.00073308  0.00245288]
622 [0.00539533] [ 0.00957467  0.00566741  0.00308283  0.00311582 -0.0006195   0.00279153]
623 [0.00545508] [ 0.01643432  0.00543297  0.00349062  0.00360037 -0.00093147  0.00224837]
624 [0.00551154] [ 0.01367097  0.00594513  0.00343972  0.00360976 -0.0005968   0.00210386]
625 [0.00554935] [ 0.01615439  0.00702976  0.00369707  0.00392322 -0.00043548  0.00202434]
626 [0.00550135] [ 0.00189812  0.00630842  0.00308236  0.0031879  -0.00077135  0.00190155]
627 [0.00553481] [ 0.01151635  0.00582131  0.00350286  0.00368112 -0.00057179  0.00178764]
628 [0.00556662] [ 0.01002286  0.00479699  0.00345171  0.00368824 -0.00059309  0.0018973 ]
629 [0.00552918] [ 0.00180522  0.00459137  0.00312809  0.0032484  -0.00098921  0.00177235]
630 [0.00557939] [ 0.01224105  0.00628928  0.00361578  0.0038406  -0.00052104  0.00168217]


709 [0.0049703] [0.01631288 0.0096029  0.00427582 0.00447736 0.00043973 0.00451737]
710 [0.00496391] [8.40336885e-03 9.04997105e-03 3.98819365e-03 4.12920148e-03
 8.48734558e-05 4.43313723e-03]
711 [0.0049683] [0.00878078 0.00913492 0.00393213 0.00407152 0.00025366 0.00442401]
712 [0.00497173] [0.0084741  0.01043119 0.00401388 0.00426004 0.0001088  0.00436208]
713 [0.00493239] [5.68610217e-03 1.01130421e-02 3.97075760e-03 3.75858134e-03
 5.78217107e-05 4.33964681e-03]
714 [0.00492475] [0.00961052 0.01000354 0.00407451 0.00381943 0.00034191 0.00431182]
715 [0.00492122] [0.01476315 0.01001538 0.00429977 0.00401949 0.00038367 0.00424769]
716 [0.00493673] [0.01747051 0.01096114 0.00445532 0.00444148 0.0003049  0.00428213]
717 [0.00495456] [0.01590916 0.01120282 0.00439691 0.00463683 0.00034513 0.00444334]
718 [0.00497859] [0.01941877 0.01113321 0.00452007 0.00498087 0.00046979 0.00440839]
719 [0.00498811] [0.02104453 0.01281107 0.00470752 0.00535841 0.00043166 0.00446958]
720 [0.00499287] 

## Forecast evaluation based on MSFE, 1957:01-2010:12

In [18]:
beta_ECON = beta_ECON[P_0:,:,:]
actual = Y[R+P_0:]
FC_HA = FC_HA[P_0:]
FC_ECON = FC_ECON[P_0:,:]
FC_ECON_CT = FC_ECON_CT[P_0:,:]
FC_OTHER = FC_OTHER[P_0:,:]
FC_OTHER_CT = FC_OTHER_CT[P_0:,:]
omega_DMSFE = omega_DMSFE[P_0:,:]
REC = data_m.iloc[1032:1680, 18].values # recession dummies, 1957:01-2010:12
EXP = 1 - REC
index_EXP = np.where(EXP)[0]
index_REC = np.where(REC)[0]
e_HA = actual.reshape(actual.shape[0],1) - FC_HA
e_ECON = np.kron(np.ones((FC_ECON.shape[1], 1)), actual).T - FC_ECON
e_ECON_CT = np.kron(np.ones((FC_ECON_CT.shape[1], 1)), actual).T - FC_ECON_CT
CSFE_HA = np.cumsum(e_HA**2)
CSFE_ECON = np.cumsum(e_ECON**2, axis=0)
CSFE_ECON_CT = np.cumsum(e_ECON_CT**2, axis=0)
DCSFE_ECON = np.kron(np.ones((FC_ECON.shape[1], 1)), CSFE_HA).T - CSFE_ECON
DCSFE_ECON_CT = np.kron(np.ones((FC_ECON_CT.shape[1], 1)), CSFE_HA).T - CSFE_ECON_CT
R2OS_ECON = np.zeros((FC_ECON.shape[1], 6))
R2OS_ECON_CT = np.zeros((FC_ECON_CT.shape[1], 6))

In [19]:
for i in range(R2OS_ECON.shape[0]):

    # Overall

    R2OS_ECON[i, 0] = 100*(1 - np.sum(e_ECON[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_ECON[:, i:i+1]**2 - (FC_HA - FC_ECON[:, i:i+1])**2)
    results_i = nwest(f_i.flatten() , np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_ECON_CT[i, 0] = 100*(1 - np.sum(e_ECON_CT[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_ECON_CT[:, i:i+1]**2 - (FC_HA - FC_ECON_CT[:, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON_CT[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Expansion
    
    R2OS_ECON[i, 2] = 100 * (1 - (np.sum(e_ECON[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_ECON[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_ECON[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_ECON_CT[i, 2] = 100 * (1 - (np.sum(e_ECON_CT[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_ECON_CT[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_ECON_CT[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON_CT[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Recession

    R2OS_ECON[i, 4] = 100 * (1 - (np.sum(e_ECON[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_ECON[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_ECON[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_ECON_CT[i, 4] = 100 * (1 - (np.sum(e_ECON_CT[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_ECON_CT[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_ECON_CT[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON_CT[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)

C:\Users\zhou\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in true_divide
C:\Users\zhou\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\zhou\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\zhou\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in true_divide
C:\Users\zhou\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\zhou\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\zhou\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarra

In [20]:
e_OTHER = np.kron(np.ones((FC_OTHER.shape[1], 1)), actual).T - FC_OTHER
e_OTHER_CT = np.kron(np.ones((FC_OTHER_CT.shape[1], 1)), actual).T - FC_OTHER_CT
CSFE_OTHER = np.cumsum(e_OTHER**2, axis=0)
CSFE_OTHER_CT = np.cumsum(e_OTHER_CT**2, axis=0)
DCSFE_OTHER = np.kron(np.ones((FC_OTHER.shape[1], 1)), CSFE_HA).T - CSFE_OTHER
DCSFE_OTHER_CT = np.kron(np.ones((FC_OTHER_CT.shape[1], 1)), CSFE_HA).T - CSFE_OTHER_CT
R2OS_OTHER = np.zeros((FC_OTHER.shape[1], 6))
R2OS_OTHER_CT = np.zeros((FC_OTHER_CT.shape[1], 6))

In [21]:
for i in range(R2OS_OTHER.shape[0]):

    # Overall
    R2OS_OTHER[i, 0] = 100 * (1 - np.sum(e_OTHER[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_OTHER[:, i:i+1]**2 - (FC_HA - FC_OTHER[:, i:i+1])**2)
    results_i = nwest(f_i.flatten() , np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_OTHER_CT[i, 0] = 100 * (1 - np.sum(e_OTHER_CT[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_OTHER_CT[:, i:i+1]**2 - (FC_HA - FC_OTHER_CT[:, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER_CT[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Expansion
    
    R2OS_OTHER[i, 2] = 100 * (1 - (np.sum(e_OTHER[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_OTHER[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_OTHER[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_OTHER_CT[i, 2] = 100 * (1 - (np.sum(e_OTHER_CT[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_OTHER_CT[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_OTHER_CT[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER_CT[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Recession
    
    R2OS_OTHER[i, 4] = 100 * (1 - (np.sum(e_OTHER[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_OTHER[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_OTHER[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_OTHER_CT[i, 4] = 100 * (1 - (np.sum(e_OTHER_CT[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_OTHER_CT[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_OTHER_CT[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER_CT[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    

## Writing results to spreadsheet

In [22]:
wb = xw.Book('Returns_handbook_results.xlsx')
sht = wb.sheets['Monthly']
sht.range('B5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON[:,0:2])
sht.range('N5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON_CT[:,0:2])
sht.range('B20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER[:,0:2])
sht.range('N20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER_CT[:,0:2])
sht.range('F5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON[:,2:4])
sht.range('R5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON_CT[:,2:4])
sht.range('F20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER[:,2:4])
sht.range('R20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER_CT[:,2:4])
sht.range('J5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON[:,4:6])
sht.range('V5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON_CT[:,4:6])
sht.range('J20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER[:,4:6])
sht.range('V20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER_CT[:,4:6])
                                                                                                   

FileNotFoundError: No such file: 'returns_handbook_results.xlsx'

# Forecast Monthly

## Loading data and defining variables, 1926:12-2010:12

In [ ]:
data_m = pd.read_excel('Returns_handbook_data.xls', sheet_name='Monthly')

In [ ]:
data_m

In [ ]:
# Equity premium

market_return = data_m.iloc[671:1680, 15].values  # S&P 500 VW return
r_f_lag = data_m.iloc[670:1679, 10].values # risk-free rate, lagged (1926:11-2010:11)
equity_premium = market_return - r_f_lag # excess return

In [ ]:
# Predictors

D12 = data_m.iloc[671:1680, 2].values # dividends
SP500 = data_m.iloc[671:1680, 1].values  # S&P 500 index
DP = np.log(D12) - np.log(SP500)  # log dividend-price ratio
SP500_lag = data_m.iloc[670:1679, 1].values  # S&P 500 index, lagged (1926:11-2010:11)
DY = np.log(D12) - np.log(SP500_lag)  # log dividend yield
E12 = data_m.iloc[671:1680, 3].values  # earnings
EP = np.log(E12) - np.log(SP500)  # log earnings-price ratio
DE = np.log(D12) - np.log(E12)  # log dividend-payout ratio
SVAR = data_m.iloc[671:1680, 14].values  # volatility
BM = data_m.iloc[671:1680, 4].values  # book-to-market ratio
NTIS = data_m.iloc[671:1680, 9].values  # net equity issuing activity
TBL = data_m.iloc[671:1680, 5].values  # T-bill rate
LTY = data_m.iloc[671:1680, 8].values  # long-term government bond yield
LTR = data_m.iloc[671:1680, 12].values  # long-term government bond return
TMS = LTY - TBL  # term spread
AAA = data_m.iloc[671:1680, 6].values  # AAA-rated corporate bond yield
BAA = data_m.iloc[671:1680, 7].values  # BAA-rated corporate bond yield
DFY = BAA - AAA  # default yield spread
CORPR = data_m.iloc[671:1680, 13].values  # long-term corporate bond return
DFR = CORPR - LTR  # default return spread
INFL_lag = data_m.iloc[670:1679, 11].values  # inflation, lagged (1926:11-2010:11)
ECON = np.column_stack((DP, DY, EP, DE, SVAR, BM, NTIS, TBL, LTY, LTR, TMS, DFY, DFR, INFL_lag))
ECON_sink = np.column_stack((DP, DY, EP, SVAR, BM, NTIS, TBL, LTY, LTR, DFY, DFR, INFL_lag))

In [ ]:
# Sum-of-the-parts variables

E12_lag = data_m.iloc[670:1679, 3].values  # earnings, lagged (1926:11-2010:11)
E_growth = (E12 - E12_lag) / E12_lag # earnings growth
DP_SOP = (1 / 12) * D12 / SP500 # D/P
r_f = data_m.iloc[671:1680, 10].values # risk-free rate

## Estimating full-sample parameters for Campbell-Thompson restrictions

In [ ]:
beta_full = np.zeros(np.shape(ECON)[1])
T = np.shape(equity_premium)[0]
LHS = equity_premium[1:T]

In [ ]:
for i in range(ECON.shape[1]):
    RHS_i = np.hstack((ECON[0:T-1, i].reshape(-1, 1), np.ones((T-1, 1))))
    results_i = ols(LHS, RHS_i)
    beta_full[i] = results_i['beta'][0]
    print(i)
beta_full[4] = 1 # restricting SVAR slope coefficient to be positive

# NB: following Campbell and Thompson (2008) by using full-sample
# estimates as theoretical 'priors' (although we make sure that
# SVAR slope coefficient is positive)

## Preliminaries

In [25]:
Y = equity_premium
T = np.shape(Y)[0]
N = np.shape(ECON)[1]
R = (1946 - 1926) * 12 + 1 # in-sample period, 1926:12-1946:12
P_0 = (1956 - 1946) * 12 # holdout out-of-sample period, 1947:01-1956:12
P = T - (R + P_0) # forecast evaluation period, 1957:01-2010:12
theta = 0.75 # discound factor for DMSFE pooled forecast
r = 1 # number of principal components
MA_SOP = 20 * 12 # window size for SOP forecast

## Computing forecasts, 1947:01-2010:12 (includes holdout OOS period)

In [26]:
FC_HA = np.zeros((P_0 + P, 1))
FC_ECON = np.zeros((P_0 + P, N))
beta_ECON = np.zeros((P_0 + P, N, 2))
FC_ECON_CT = np.zeros((P_0 + P, N))
omega_DMSFE = np.zeros((P_0 + P, N))
FC_OTHER = np.zeros((P_0 + P, 6))
FC_OTHER_CT = np.zeros((P_0 + P, 6))

In [27]:
for t in range(P_0 + P):
    
    FC_HA[t] = np.mean(Y[0:R+t])
    
    # Individual predictive regression forecasts

    X_t = ECON[0:R+t-1,:]
    Y_t = Y[1:R+t]
    for i in range(N):
        results_t_i = ols(Y_t, np.hstack((X_t[:,i:i+1], np.ones((R+t-1,1)))))
        FC_ECON[t,i] = np.dot(np.array([[ECON[R+(t-1),i], 1]]), results_t_i['beta']) # scaler output
        beta_ECON[t,i,0] = results_t_i['beta'][0]
        beta_ECON[t,i,1] = results_t_i['bstd'][0]
        if beta_full[i] > 0:
            if results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        elif beta_full[i] < 0:
            if results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        if FC_ECON_CT[t,i] < 0:
            FC_ECON_CT[t,i] = 0
    if t >= P_0:

        # Kitchen sink forecast

        X_t_sink = ECON_sink[0:R+t-1,:]
        results_t_sink = ols(Y_t, np.hstack((X_t_sink, np.ones((R+t-1,1)))))
        FC_OTHER[t,0] = np.dot(np.append(ECON_sink[R+(t-1),:], 1), results_t_sink['beta'])
        if FC_OTHER[t,0] < 0:
            FC_OTHER_CT[t,0] = 0
        else:
            FC_OTHER_CT[t,0] = FC_OTHER[t,0]
        # SIC forecast
        
        j_max = 3  # consider models with up to 3 predictors to save time
        SIC_t = np.empty((0, 2))
        for j in range(1, j_max + 1):
            select_j = list(itertools.combinations(range(X_t_sink.shape[1]), j))
            for k in range(len(select_j)):
                X_t_j_k = np.hstack((X_t_sink[0:R+t-1,select_j[k]], np.ones((R+t-1,1))))
                results_t_j_k = ols(Y_t, X_t_j_k)
                SIC_t_j_k = np.log(np.dot(results_t_j_k['resid'].T, results_t_j_k['resid']) / np.shape(Y_t)[0]) + \
                    np.log(np.shape(Y_t)[0]) * np.shape(X_t_j_k)[1] / np.shape(Y_t)[0]
                FC_t_j_k = np.hstack((ECON_sink[R+(t-1),select_j[k]], 1)).dot(results_t_j_k['beta'])
                SIC_t = np.vstack((SIC_t, [SIC_t_j_k, FC_t_j_k]))
        SIC_t_min_index = np.argmin(SIC_t[:,0])
        SIC_t_min = SIC_t[SIC_t_min_index,0]
        FC_OTHER[t,1] = SIC_t[SIC_t_min_index,1];
        if FC_OTHER[t,1] < 0:
            FC_OTHER_CT[t,1] = 0
        else:
            FC_OTHER_CT[t,1] = FC_OTHER[t,1]
       
        # Pooled forecast: simple average

        FC_OTHER[t,2] = np.mean(FC_ECON[t,:])
        if FC_OTHER[t,2] < 0:
            FC_OTHER_CT[t,2] = 0
        else:
            FC_OTHER_CT[t,2] = FC_OTHER[t,2]
            
        # Pooled forecast: DMSFE

        powers_t = np.arange(t-1, -1, -1).reshape(-1, 1)
        m = np.sum((np.kron(np.ones((1, N)), (theta*np.ones((t, 1))))**powers_t) * 
                   (((np.kron(Y[R:R+t], np.ones((1, N))).reshape(t, N) - FC_ECON[0:t, :])**2)), axis=0)
        omega = m**(-1) / np.sum(m**(-1))
        FC_OTHER[t, 3] = FC_ECON[t, :].dot(omega)
        if FC_OTHER[t,3] < 0:
            FC_OTHER_CT[t,3] = 0
        else:
            FC_OTHER_CT[t,3] = FC_OTHER[t,3]
        
        # Diffusion index forecast

        X_t_DI_standardize = zscore(ECON[0:R+t, :])
        pca = PCA()
        pca.fit(X_t_DI_standardize)
        Lambda_t = pca.components_.T
        F_t = pca.transform(X_t_DI_standardize)
        results_t_DI = ols(Y_t, np.hstack((F_t[0:-1, 0:r], np.ones((R+t-1,1)))))
        FC_OTHER[t,4] = np.dot(np.append(F_t[-1,0:r], 1), results_t_DI['beta'])
        if FC_OTHER[t,4] < 0:
            FC_OTHER_CT[t,4] = 0
        else:
            FC_OTHER_CT[t,4] = FC_OTHER[t,4]

        # Sum-of-the-parts forecast

        FC_OTHER[t, 5] = np.mean(E_growth[R+(t-1)-MA_SOP+1:R+t]) + DP_SOP[R+(t-1)] - r_f[R+(t-1)]

        if FC_OTHER[t,5] < 0:
            FC_OTHER_CT[t,5] = 0
        else:
            FC_OTHER_CT[t,5] = FC_OTHER[t,5]
        
    print(t, FC_HA[t], FC_OTHER[t, :])

0 [0.00357551] [0. 0. 0. 0. 0. 0.]
1 [0.00364943] [0. 0. 0. 0. 0. 0.]
2 [0.00359878] [0. 0. 0. 0. 0. 0.]
3 [0.00352056] [0. 0. 0. 0. 0. 0.]
4 [0.00334019] [0. 0. 0. 0. 0. 0.]
5 [0.00332129] [0. 0. 0. 0. 0. 0.]
6 [0.0035194] [0. 0. 0. 0. 0. 0.]
7 [0.00366043] [0. 0. 0. 0. 0. 0.]
8 [0.00354975] [0. 0. 0. 0. 0. 0.]
9 [0.00349324] [0. 0. 0. 0. 0. 0.]
10 [0.00355782] [0. 0. 0. 0. 0. 0.]
11 [0.00347028] [0. 0. 0. 0. 0. 0.]
12 [0.00357338] [0. 0. 0. 0. 0. 0.]
13 [0.00340473] [0. 0. 0. 0. 0. 0.]
14 [0.00322718] [0. 0. 0. 0. 0. 0.]
15 [0.00351716] [0. 0. 0. 0. 0. 0.]
16 [0.00361834] [0. 0. 0. 0. 0. 0.]
17 [0.00391378] [0. 0. 0. 0. 0. 0.]
18 [0.00392494] [0. 0. 0. 0. 0. 0.]
19 [0.00370024] [0. 0. 0. 0. 0. 0.]
20 [0.00374186] [0. 0. 0. 0. 0. 0.]
21 [0.00361981] [0. 0. 0. 0. 0. 0.]
22 [0.00385972] [0. 0. 0. 0. 0. 0.]
23 [0.00344942] [0. 0. 0. 0. 0. 0.]
24 [0.00356611] [0. 0. 0. 0. 0. 0.]
25 [0.00355912] [0. 0. 0. 0. 0. 0.]
26 [0.00342172] [0. 0. 0. 0. 0. 0.]
27 [0.00353001] [0. 0. 0. 0. 0. 0.]
28 

166 [0.00629191] [-0.00289312  0.0094448   0.00428241  0.00401572  0.0019171   0.0057789 ]
167 [0.00638185] [-0.00069802  0.00934669  0.00421831  0.00418768  0.0017371   0.00564233]
168 [0.0064778] [-0.00654467  0.00941549  0.00366736  0.00379351  0.00215665  0.00565261]
169 [0.00660928] [0.00751084 0.00858467 0.00449588 0.00476406 0.00167874 0.00540042]
170 [0.00666719] [-0.00345365  0.00793872  0.00376841  0.00415475  0.00168433  0.00508599]
171 [0.00671624] [0.00565039 0.01108037 0.00405171 0.00453677 0.00162137 0.00495004]
172 [0.00670619] [-0.00346947  0.01064301  0.00352712  0.00399558  0.00183509  0.00495755]
173 [0.0067414] [0.00418838 0.00986379 0.00409003 0.00470988 0.00157606 0.00484518]
174 [0.00665541] [-0.00362946  0.00902617  0.00388281  0.00402523  0.00141547  0.00481806]
175 [0.00671456] [-0.00061855  0.00870397  0.00375974  0.0041568   0.00162076  0.00470566]
176 [0.00675266] [-0.00153168  0.00840959  0.00377087  0.00433996  0.00128132  0.00442965]
177 [0.00668664] [-

260 [0.00627865] [0.00710792 0.01104032 0.00444323 0.00447571 0.00044953 0.0026193 ]
261 [0.0063346] [0.01220983 0.01009911 0.00445431 0.00468482 0.00047783 0.00233179]
262 [0.00633042] [0.00776329 0.00948367 0.00432458 0.00455932 0.00023943 0.00207161]
263 [0.0064126] [0.01365535 0.009328   0.00446201 0.00489805 0.00028226 0.00177105]
264 [0.00630839] [ 0.00555433  0.00975335  0.00414331  0.00410031 -0.00060405  0.00134912]
265 [0.00627116] [ 0.00342222  0.00930975  0.00395796  0.00381062 -0.00077551  0.00118843]
266 [0.00616343] [-0.00884768  0.00979617  0.0033441   0.00303827 -0.00067058  0.00129794]
267 [0.00620871] [ 0.00486305  0.01284276  0.00363085  0.00351229 -0.00025336  0.00126338]
268 [0.00623092] [-0.00371833  0.01244247  0.00323832  0.00321961 -0.00026384  0.00113652]
269 [0.00621628] [ 0.01022114  0.01273452  0.00392543  0.00383581 -0.00074781  0.00120128]
270 [0.00608492] [-0.0046629   0.01425911  0.00333137  0.00305441 -0.00067809  0.00101885]
271 [0.00594457] [-0.0006

352 [0.00503956] [0.00011996 0.00936495 0.00503041 0.00523764 0.00264461 0.00268018]
353 [0.00501055] [-0.00442376  0.00995927  0.00481427  0.00493478  0.00207762  0.00258742]
354 [0.00506716] [0.00488846 0.00921098 0.00495828 0.00525592 0.00179226 0.00243406]
355 [0.00503899] [0.00204549 0.00969692 0.00496384 0.00514542 0.0018449  0.002731  ]
356 [0.00502594] [0.00374269 0.00999662 0.00509783 0.00521306 0.00200056 0.00294497]
357 [0.00505073] [0.00623285 0.00952681 0.00502811 0.0052885  0.00184066 0.00304985]
358 [0.00500116] [0.00064011 0.01021138 0.00503089 0.00495038 0.00213953 0.00337772]
359 [0.00498427] [0.00724797 0.01073045 0.0053799  0.00518946 0.00257801 0.00366943]
360 [0.00505656] [0.01510427 0.00908482 0.0055142  0.00575049 0.0028882  0.00396247]
361 [0.00495755] [-0.00357843  0.01045427  0.00481435  0.00469457  0.00254958  0.00397074]
362 [0.00491681] [-0.00351845  0.01095207  0.00512724  0.00486447  0.00288041  0.00406958]
363 [0.00488141] [-0.00379974  0.01202501  0.00

444 [0.0046169] [-0.0240069   0.0044117   0.00397624  0.00394555  0.00280328  0.0013119 ]
445 [0.00459358] [-0.01594125  0.00500051  0.00432922  0.00413415  0.00346569  0.00150816]
446 [0.00452574] [-0.02544214  0.00608653  0.00399658  0.00366547  0.00360463  0.00157557]
447 [0.00453311] [-0.02207852  0.00606474  0.00402823  0.00377208  0.00348399  0.00124529]
448 [0.00452487] [-0.0184563   0.00594678  0.00425262  0.00394151  0.00367819  0.001142  ]
449 [0.00442754] [-0.03322201 -0.01176185  0.00395409  0.0034749   0.00406052  0.00132063]
450 [0.00444214] [-0.01643073 -0.01191533  0.00506258  0.00472548  0.00512603  0.00127524]
451 [0.00440386] [-0.01300192 -0.00989336  0.00528825  0.00474936  0.00524228  0.00113381]
452 [0.00453914] [-0.00276305  0.00501992  0.00494415  0.00512348  0.00440008  0.00045424]
453 [0.00451999] [-0.01021643  0.00529813  0.00492175  0.00507124  0.00373048  0.00054788]
454 [0.00450589] [-0.00565754  0.00527118  0.00526842  0.005383    0.00370281  0.00107566]


534 [0.00479086] [-0.01639999 -0.00124863  0.0032172   0.00318143  0.00109247  0.00348627]
535 [0.00483732] [-0.00907191  0.00639582  0.00330989  0.00338905  0.00111593  0.00322648]
536 [0.00485551] [-0.00961035  0.00582988  0.00327288  0.0034166   0.0012939   0.00324556]
537 [0.00482171] [-0.01422085 -0.00157053  0.00308165  0.00308219  0.00104559  0.00324596]
538 [0.00482711] [-0.01743342  0.0039251   0.00280371  0.00284708  0.00113983  0.00318509]
539 [0.00476298] [-0.01873587 -0.00117606  0.00292203  0.00273584  0.00187144  0.00343583]
540 [0.00489036] [-0.0063016   0.00255098  0.0030605   0.00319098  0.00175802  0.00334132]
541 [0.00485609] [-0.01839832  0.00248505  0.00270779  0.00278534  0.00138359  0.00361546]
542 [0.00486218] [-0.0141494   0.00212157  0.00286808  0.00295927  0.00146344  0.00355917]
543 [0.00482682] [-0.02273567 -0.00248597  0.00246423  0.00242802  0.0011088   0.00344902]
544 [0.00485233] [-0.01733491  0.00094752  0.00263732  0.00272057  0.00124497  0.00365613]

622 [0.00539533] [ 0.00957467  0.00566741  0.00308283  0.00311582 -0.0006195   0.00279153]
623 [0.00545508] [ 0.01643432  0.00543297  0.00349062  0.00360037 -0.00093147  0.00224837]
624 [0.00551154] [ 0.01367097  0.00594513  0.00343972  0.00360976 -0.0005968   0.00210386]
625 [0.00554935] [ 0.01615439  0.00702976  0.00369707  0.00392322 -0.00043548  0.00202434]
626 [0.00550135] [ 0.00189812  0.00630842  0.00308236  0.0031879  -0.00077135  0.00190155]
627 [0.00553481] [ 0.01151635  0.00582131  0.00350286  0.00368112 -0.00057179  0.00178764]
628 [0.00556662] [ 0.01002286  0.00479699  0.00345171  0.00368824 -0.00059309  0.0018973 ]
629 [0.00552918] [ 0.00180522  0.00459137  0.00312809  0.0032484  -0.00098921  0.00177235]
630 [0.00557939] [ 0.01224105  0.00628928  0.00361578  0.0038406  -0.00052104  0.00168217]
631 [0.00553344] [ 0.00897734  0.00611127  0.00359992  0.00366431 -0.00084814  0.00177005]
632 [0.00551704] [ 0.01042001  0.00620628  0.00359359  0.00360843 -0.00089421  0.00167245]

712 [0.00497173] [0.0084741  0.01043119 0.00401388 0.00426004 0.0001088  0.00436208]
713 [0.00493239] [5.68610217e-03 1.01130421e-02 3.97075760e-03 3.75858134e-03
 5.78217107e-05 4.33964681e-03]
714 [0.00492475] [0.00961052 0.01000354 0.00407451 0.00381943 0.00034191 0.00431182]
715 [0.00492122] [0.01476315 0.01001538 0.00429977 0.00401949 0.00038367 0.00424769]
716 [0.00493673] [0.01747051 0.01096114 0.00445532 0.00444148 0.0003049  0.00428213]
717 [0.00495456] [0.01590916 0.01120282 0.00439691 0.00463683 0.00034513 0.00444334]
718 [0.00497859] [0.01941877 0.01113321 0.00452007 0.00498087 0.00046979 0.00440839]
719 [0.00498811] [0.02104453 0.01281107 0.00470752 0.00535841 0.00043166 0.00446958]
720 [0.00499287] [0.01215484 0.01270673 0.0043249  0.00496914 0.00037629 0.00462714]
721 [0.00499932] [0.01353534 0.01362585 0.00443732 0.00525604 0.00032715 0.00447956]
722 [0.00496962] [0.01272025 0.0125265  0.00444308 0.00431352 0.00030986 0.00445197]
723 [0.00497137] [0.005584   0.01185315 

## Forecast evaluation based on asselt allocation exercise, 1957:01-2010:12

In [28]:
actual = Y[R+P_0:]
FC_HA = FC_HA[P_0:]
FC_ECON = FC_ECON[P_0:,:]
FC_ECON_CT = FC_ECON_CT[P_0:,:]
FC_OTHER = FC_OTHER[P_0:,:]
FC_OTHER_CT = FC_OTHER_CT[P_0:,:]
REC = data_m.iloc[1032:1680, 18].values # recession dummies, 1957:01-2010:12
EXP = 1 - REC
index_EXP = np.where(EXP)[0]
index_REC = np.where(REC)[0]
gamma_MV = 5 # coefficient of relative risk aversion
window_VOL = 12 * 5 # window size for estimating volatility
FC_VOL = np.zeros((P,1))

In [29]:
for t in range(P):
    FC_VOL[t,:] = np.mean(Y[R+P_0+(t-1)-window_VOL+1: R+P_0+t]**2) \
        - (np.mean(Y[R+P_0+(t-1)-window_VOL+1: R+P_0+t]))**2

In [30]:
r_f_lag_P = r_f_lag[R+P_0: R+P_0+P]
U_HA = np.zeros((3,1))
U_HA[0,:], w_HA = Perform_asset_allocation(actual, r_f_lag_P, FC_HA,FC_VOL, gamma_MV)
U_HA[1,:], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_HA[index_EXP], 
                                          FC_VOL[index_EXP], gamma_MV)
U_HA[2,:], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_HA[index_REC], 
                                          FC_VOL[index_REC], gamma_MV)
U_ECON = np.zeros((FC_ECON.shape[1] ,3))
U_ECON_CT = np.zeros((FC_ECON.shape[1] ,3))
w_ECON = np.zeros((P, FC_ECON.shape[1]))
w_ECON_CT = np.zeros((P, FC_ECON_CT.shape[1]))

In [31]:
for i in range(U_ECON.shape[0]):

    # Overall

    U_ECON[i,0], w_ECON[:,i] = Perform_asset_allocation(actual, r_f_lag_P,FC_ECON[:,i], FC_VOL, gamma_MV)
    U_ECON_CT[i,0], w_ECON_CT[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_ECON_CT[:,i], FC_VOL, gamma_MV)

    # Expansion

    U_ECON[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_ECON[index_EXP,i], 
                                                FC_VOL[index_EXP], gamma_MV)
    U_ECON_CT[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_ECON_CT[index_EXP,i], 
                                                   FC_VOL[index_EXP], gamma_MV)

    # Recession

    U_ECON[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_ECON[index_REC,i], 
                                                FC_VOL[index_REC], gamma_MV)
    U_ECON_CT[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_ECON_CT[index_REC,i], 
                                                   FC_VOL[index_REC], gamma_MV)

In [32]:
delta_ECON = 1200 * (U_ECON - np.kron(U_HA.T, np.ones((FC_ECON.shape[1],1))))
delta_ECON_CT = 1200 * (U_ECON_CT - np.kron(U_HA.T, np.ones((FC_ECON_CT.shape[1],1))))
U_OTHER = np.zeros((FC_OTHER.shape[1], 3))
U_OTHER_CT = np.zeros((FC_OTHER_CT.shape[1], 3))
w_OTHER = np.zeros((P, FC_OTHER.shape[1]))
w_OTHER_CT = np.zeros((P, FC_OTHER_CT.shape[1]))

In [33]:
for i in range(U_OTHER.shape[0]):

    # Overall

    U_OTHER[i,0], w_OTHER[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_OTHER[:,i], FC_VOL,gamma_MV)
    U_OTHER_CT[i,0], w_OTHER_CT[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_OTHER_CT[:,i], FC_VOL,gamma_MV)

    # Expansion

    U_OTHER[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_OTHER[index_EXP,i],
                                                 FC_VOL[index_EXP], gamma_MV)
    U_OTHER_CT[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_OTHER_CT[index_EXP,i],
                                                   FC_VOL[index_EXP], gamma_MV)

    # Recession

    U_OTHER[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_OTHER[index_REC,i],
                                                 FC_VOL[index_REC], gamma_MV)
    U_OTHER_CT[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_OTHER_CT[index_REC,i],
                                                    FC_VOL[index_REC], gamma_MV)

In [34]:
delta_OTHER = 1200 * (U_OTHER - np.kron(U_HA.T, np.ones((FC_OTHER.shape[1], 1))))
delta_OTHER_CT = 1200 * (U_OTHER_CT - np.kron(U_HA.T, np.ones((FC_OTHER_CT.shape[1], 1))))

## Writing results to spreadsheet

In [35]:
wb = xw.Book('Returns_handbook_results.xlsx')
sht = wb.sheets['Monthly']
sht.range('D5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON[:,0:1])
sht.range('D20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER[:,0:1])
sht.range('P5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON_CT[:,0:1])
sht.range('P20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER_CT[:,0:1])
sht.range('H5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON[:,1:2])
sht.range('H20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER[:,1:2])
sht.range('T5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON_CT[:,1:2])
sht.range('T20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER_CT[:,1:2])
sht.range('L5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON[:,2:3])
sht.range('L20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER[:,2:3])
sht.range('X5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON_CT[:,2:3])
sht.range('X20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER_CT[:,2:3])
   

FileNotFoundError: No such file: 'returns_handbook_results.xlsx'

# Forecasts Quarterly Log

## Loading data and defining variables, 1926:4-2010:4

In [ ]:
# load quarterly data into pandas dataframe
data_q = pd.read_excel('Returns_handbook_data.xls', sheet_name='Quarterly')

In [ ]:
data_q

In [ ]:
# Equity premium

market_return = data_q.iloc[223:560, 15].values  # S&P 500 VW returns
r_f_lag = data_q.iloc[222:559, 10].values # risk-free rate, lagged (1926:3-2010:3)
equity_premium = np.log(1 + market_return) - np.log(1 + r_f_lag) # log excess return

In [ ]:
# Predictors

D4 = data_q.iloc[223:560, 2].values # dividends
SP500 = data_q.iloc[223:560, 1].values  # S&P 500 index
DP = np.log(D4) - np.log(SP500)  # log dividend-price ratio
SP500_lag = data_q.iloc[222:559, 1].values  # S&P 500 index, lagged (1926:3-2010:3)
DY = np.log(D4) - np.log(SP500_lag)  # log dividend yield
E4 = data_q.iloc[223:560, 3].values  # earnings
EP = np.log(E4) - np.log(SP500)  # log earnings-price ratio
DE = np.log(D4) - np.log(E4)  # log dividend-payout ratio
SVAR = data_q.iloc[223:560, 14].values  # volatility
BM = data_q.iloc[223:560, 4].values  # book-to-market ratio
NTIS = data_q.iloc[223:560, 9].values  # net equity issuing activity
TBL = data_q.iloc[223:560, 5].values  # T-bill rate
LTY = data_q.iloc[223:560, 8].values  # long-term government bond yield
LTR = data_q.iloc[223:560, 12].values  # long-term government bond return
TMS = LTY - TBL  # term spread
AAA = data_q.iloc[223:560, 6].values  # AAA-rated corporate bond yield
BAA = data_q.iloc[223:560, 7].values  # BAA-rated corporate bond yield
DFY = BAA - AAA  # default yield spread
CORPR = data_q.iloc[223:560, 13].values  # long-term corporate bond return
DFR = CORPR - LTR  # default return spread
INFL_lag = data_q.iloc[222:559, 11].values  # inflation, lagged (1926:3-2010:3)
ECON = np.column_stack((DP, DY, EP, DE, SVAR, BM, NTIS, TBL, LTY, LTR, TMS, DFY, DFR, INFL_lag))
ECON_sink = np.column_stack((DP, DY, EP, SVAR, BM, NTIS, TBL, LTY, LTR, DFY, DFR, INFL_lag))

In [ ]:
# Sum-of-the-parts variables

E4_lag = data_q.iloc[222:559, 3].values  # earnings, lagged (1926:3-2010:3)
E_growth = np.log((1 / 4) * E4) - np.log((1 / 4) * E4_lag) # earnings growth
DP_SOP = np.log(1 + (1 / 4) * np.divide(D4, SP500)) # log (1+D/P)
r_f = data_q.iloc[223:560, 10].values # risk-free rate
r_f = np.log(1 + r_f) # log risk-free rate

## Estimating full-sample parameters for Campbell-Thompson restrictions

In [ ]:
beta_full = np.zeros(np.shape(ECON)[1])
T = np.shape(equity_premium)[0]
LHS = equity_premium[1:T]

In [ ]:
for i in range(ECON.shape[1]):
    RHS_i = np.hstack((ECON[0:T-1, i].reshape(-1, 1), np.ones((T-1, 1))))
    results_i = ols(LHS, RHS_i)
    beta_full[i] = results_i['beta'][0]
    print(i)
beta_full[4] = 1 # restricting SVAR slope coefficient to be positive

# NB: following Campbell and Thompson (2008) by using full-sample
# estimates as theoretical 'priors' (although we make sure that
# SVAR slope coefficient is positive)

## Preliminaries

In [ ]:
Y = equity_premium
T = np.shape(Y)[0]
N = np.shape(ECON)[1]
R = (1946 - 1926) * 4 + 1 # in-sample period, 1926:4-1946:4
P_0 = (1956 - 1946) * 4 # holdout out-of-sample period, 1947:1-1956:4
P = T - (R + P_0) # forecast evaluation period, 1957:1-2010:4
theta = 0.75 # discound factor for DMSFE pooled forecast
r = 1 # number of principal components
MA_SOP = 20 * 4 # window size for SOP forecast

## Computing forecasts, 1947:1-2010:4 (includes holdout OOS period)

In [ ]:
FC_HA = np.zeros((P_0 + P, 1))
FC_ECON = np.zeros((P_0 + P, N))
beta_ECON = np.zeros((P_0 + P, N, 2))
FC_ECON_CT = np.zeros((P_0 + P, N))
omega_DMSFE = np.zeros((P_0 + P, N))
FC_OTHER = np.zeros((P_0 + P, 6))
FC_OTHER_CT = np.zeros((P_0 + P, 6))

In [ ]:
for t in range(P_0 + P):

    FC_HA[t] = np.mean(Y[0:R+t])
    
    # Individual predictive regression forecasts

    X_t = ECON[0:R+t-1,:]
    Y_t = Y[1:R+t]
    for i in range(N):
        results_t_i = ols(Y_t, np.hstack((X_t[:,i:i+1], np.ones((R+t-1,1)))))
        FC_ECON[t,i] = np.dot(np.array([[ECON[R+(t-1),i], 1]]), results_t_i['beta']) # scaler output
        beta_ECON[t,i,0] = results_t_i['beta'][0]
        beta_ECON[t,i,1] = results_t_i['bstd'][0]
        if beta_full[i] > 0:
            if results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        elif beta_full[i] < 0:
            if results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        if FC_ECON_CT[t,i] < 0:
            FC_ECON_CT[t,i] = 0
    if t >= P_0:

        # Kitchen sink forecast

        X_t_sink = ECON_sink[0:R+t-1,:]
        results_t_sink = ols(Y_t, np.hstack((X_t_sink, np.ones((R+t-1,1)))))
        FC_OTHER[t,0] = np.dot(np.append(ECON_sink[R+(t-1),:], 1), results_t_sink['beta'])
        if FC_OTHER[t,0] < 0:
            FC_OTHER_CT[t,0] = 0
        else:
            FC_OTHER_CT[t,0] = FC_OTHER[t,0]
        
        # SIC forecast
        
        j_max = 3  # consider models with up to 3 predictors to save time
        SIC_t = np.empty((0, 2))
        for j in range(1, j_max + 1):
            select_j = list(itertools.combinations(range(X_t_sink.shape[1]), j))
            for k in range(len(select_j)):
                X_t_j_k = np.hstack((X_t_sink[0:R+t-1,select_j[k]], np.ones((R+t-1,1))))
                results_t_j_k = ols(Y_t, X_t_j_k)
                SIC_t_j_k = np.log(np.dot(results_t_j_k['resid'].T, results_t_j_k['resid']) / np.shape(Y_t)[0]) + \
                    np.log(np.shape(Y_t)[0]) * np.shape(X_t_j_k)[1] / np.shape(Y_t)[0]
                FC_t_j_k = np.hstack((ECON_sink[R+(t-1),select_j[k]], 1)).dot(results_t_j_k['beta'])
                SIC_t = np.vstack((SIC_t, [SIC_t_j_k, FC_t_j_k]))
        SIC_t_min_index = np.argmin(SIC_t[:,0])
        SIC_t_min = SIC_t[SIC_t_min_index,0]
        FC_OTHER[t,1] = SIC_t[SIC_t_min_index,1];
        if FC_OTHER[t,1] < 0:
            FC_OTHER_CT[t,1] = 0
        else:
            FC_OTHER_CT[t,1] = FC_OTHER[t,1]

        # Pooled forecast: simple average

        FC_OTHER[t,2] = np.mean(FC_ECON[t,:])
        if FC_OTHER[t,2] < 0:
            FC_OTHER_CT[t,2] = 0
        else:
            FC_OTHER_CT[t,2] = FC_OTHER[t,2] 

        # Pooled forecast: DMSFE

        powers_t = np.arange(t-1, -1, -1).reshape(-1, 1)
        m = np.sum((np.kron(np.ones((1, N)), (theta*np.ones((t, 1))))**powers_t) * 
                   (((np.kron(Y[R:R+t], np.ones((1, N))).reshape(t, N) - FC_ECON[0:t, :])**2)), axis=0)
        omega = m**(-1) / np.sum(m**(-1))
        omega_DMSFE[t, :] = omega
        FC_OTHER[t, 3] = FC_ECON[t, :].dot(omega)
        if FC_OTHER[t,3] < 0:
            FC_OTHER_CT[t,3] = 0
        else:
            FC_OTHER_CT[t,3] = FC_OTHER[t,3]

        # Diffusion index forecast

        X_t_DI_standardize = zscore(ECON[0:R+t, :])
        pca = PCA()
        pca.fit(X_t_DI_standardize)
        Lambda_t = pca.components_.T
        F_t = pca.transform(X_t_DI_standardize)
        results_t_DI = ols(Y_t, np.hstack((F_t[0:-1, 0:r], np.ones((R+t-1,1)))))
        FC_OTHER[t,4] = np.dot(np.append(F_t[-1,0:r], 1), results_t_DI['beta'])
        if FC_OTHER[t,4] < 0:
            FC_OTHER_CT[t,4] = 0
        else:
            FC_OTHER_CT[t,4] = FC_OTHER[t,4]

        # Sum-of-the-parts forecast

        FC_OTHER[t, 5] = np.mean(E_growth[R+(t-1)-MA_SOP+1:R+t]) + DP_SOP[R+(t-1)] - r_f[R+(t-1)]

        if FC_OTHER[t,5] < 0:
            FC_OTHER_CT[t,5] = 0
        else:
            FC_OTHER_CT[t,5] = FC_OTHER[t,5]
        
    print(t, FC_HA[t], FC_OTHER[t, :])

## Forecast evaluation based on MSFE, 1957:1-2010:4

In [ ]:
beta_ECON = beta_ECON[P_0:,:,:]
actual = Y[R+P_0:]
FC_HA = FC_HA[P_0:]
FC_ECON = FC_ECON[P_0:,:]
FC_ECON_CT = FC_ECON_CT[P_0:,:]
FC_OTHER = FC_OTHER[P_0:,:]
FC_OTHER_CT = FC_OTHER_CT[P_0:,:]
omega_DMSFE = omega_DMSFE[P_0:,:]
REC = data_q.iloc[344:560, 18].values # recession dummies, 1957:1-2010:4
EXP = 1 - REC
index_EXP = np.where(EXP)[0]
index_REC = np.where(REC)[0]
e_HA = actual.reshape(actual.shape[0],1) - FC_HA
e_ECON = np.kron(np.ones((FC_ECON.shape[1], 1)), actual).T - FC_ECON
e_ECON_CT = np.kron(np.ones((FC_ECON_CT.shape[1], 1)), actual).T - FC_ECON_CT
CSFE_HA = np.cumsum(e_HA**2)
CSFE_ECON = np.cumsum(e_ECON**2, axis=0)
CSFE_ECON_CT = np.cumsum(e_ECON_CT**2, axis=0)
DCSFE_ECON = np.kron(np.ones((FC_ECON.shape[1], 1)), CSFE_HA).T - CSFE_ECON
DCSFE_ECON_CT = np.kron(np.ones((FC_ECON_CT.shape[1], 1)), CSFE_HA).T - CSFE_ECON_CT
R2OS_ECON = np.zeros((FC_ECON.shape[1], 6))
R2OS_ECON_CT = np.zeros((FC_ECON_CT.shape[1], 6))

In [ ]:
for i in range(R2OS_ECON.shape[0]):

    # Overall

    R2OS_ECON[i, 0] = 100*(1 - np.sum(e_ECON[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_ECON[:, i:i+1]**2 - (FC_HA - FC_ECON[:, i:i+1])**2)
    results_i = nwest(f_i.flatten() , np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_ECON_CT[i, 0] = 100*(1 - np.sum(e_ECON_CT[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_ECON_CT[:, i:i+1]**2 - (FC_HA - FC_ECON_CT[:, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON_CT[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Expansion
    
    R2OS_ECON[i, 2] = 100 * (1 - (np.sum(e_ECON[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_ECON[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_ECON[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_ECON_CT[i, 2] = 100 * (1 - (np.sum(e_ECON_CT[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_ECON_CT[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_ECON_CT[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON_CT[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Recession

    R2OS_ECON[i, 4] = 100 * (1 - (np.sum(e_ECON[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_ECON[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_ECON[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_ECON_CT[i, 4] = 100 * (1 - (np.sum(e_ECON_CT[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_ECON_CT[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_ECON_CT[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON_CT[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)

In [ ]:
e_OTHER = np.kron(np.ones((FC_OTHER.shape[1], 1)), actual).T - FC_OTHER
e_OTHER_CT = np.kron(np.ones((FC_OTHER_CT.shape[1], 1)), actual).T - FC_OTHER_CT
CSFE_OTHER = np.cumsum(e_OTHER**2, axis=0)
CSFE_OTHER_CT = np.cumsum(e_OTHER_CT**2, axis=0)
DCSFE_OTHER = np.kron(np.ones((FC_OTHER.shape[1], 1)), CSFE_HA).T - CSFE_OTHER
DCSFE_OTHER_CT = np.kron(np.ones((FC_OTHER_CT.shape[1], 1)), CSFE_HA).T - CSFE_OTHER_CT
R2OS_OTHER = np.zeros((FC_OTHER.shape[1], 6))
R2OS_OTHER_CT = np.zeros((FC_OTHER_CT.shape[1], 6))

In [ ]:
for i in range(R2OS_OTHER.shape[0]):

    # Overall
    R2OS_OTHER[i, 0] = 100 * (1 - np.sum(e_OTHER[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_OTHER[:, i:i+1]**2 - (FC_HA - FC_OTHER[:, i:i+1])**2)
    results_i = nwest(f_i.flatten() , np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_OTHER_CT[i, 0] = 100 * (1 - np.sum(e_OTHER_CT[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_OTHER_CT[:, i:i+1]**2 - (FC_HA - FC_OTHER_CT[:, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER_CT[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Expansion
    
    R2OS_OTHER[i, 2] = 100 * (1 - (np.sum(e_OTHER[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_OTHER[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_OTHER[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_OTHER_CT[i, 2] = 100 * (1 - (np.sum(e_OTHER_CT[index_EXP, i]**2) / np.sum(e_HA[index_EXP]**2)))
    f_i = e_HA[index_EXP]**2 - (e_OTHER_CT[index_EXP, i:i+1]**2 - (FC_HA[index_EXP] - FC_OTHER_CT[index_EXP, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER_CT[i, 3] = 1 - norm.cdf(results_i['tstat'], 0, 1)

    # Recession
    
    R2OS_OTHER[i, 4] = 100 * (1 - (np.sum(e_OTHER[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_OTHER[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_OTHER[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_OTHER_CT[i, 4] = 100 * (1 - (np.sum(e_OTHER_CT[index_REC, i]**2) / np.sum(e_HA[index_REC]**2)))
    f_i = e_HA[index_REC]**2 - (e_OTHER_CT[index_REC, i:i+1]**2 - (FC_HA[index_REC] - FC_OTHER_CT[index_REC, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER_CT[i, 5] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    

## Writing results to spreadsheet

In [ ]:
wb = xw.Book('Returns_handbook_results.xlsx')
sht = wb.sheets['Quarterly']
sht.range('B5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON[:,0:2])
sht.range('N5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON_CT[:,0:2])
sht.range('B20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER[:,0:2])
sht.range('N20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER_CT[:,0:2])
sht.range('F5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON[:,2:4])
sht.range('R5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON_CT[:,2:4])
sht.range('F20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER[:,2:4])
sht.range('R20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER_CT[:,2:4])
sht.range('J5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON[:,4:6])
sht.range('V5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON_CT[:,4:6])
sht.range('J20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER[:,4:6])
sht.range('V20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER_CT[:,4:6])
                                                                                                   

# Forecasts Quarterly

## Loading data and defining variables, 1926:4-2010:4

In [ ]:
data_q = pd.read_excel('Returns_handbook_data.xls', sheet_name='Quarterly')

In [ ]:
data_q

In [ ]:
# Equity premium

market_return = data_q.iloc[223:560, 15].values  # S&P 500 VW return
r_f_lag = data_q.iloc[222:559, 10].values # risk-free rate, lagged (1926:3-2010:3)
equity_premium = market_return - r_f_lag # excess return

In [ ]:
# Predictors

D4 = data_q.iloc[223:560, 2].values # dividends
SP500 = data_q.iloc[223:560, 1].values  # S&P 500 index
DP = np.log(D4) - np.log(SP500)  # log dividend-price ratio
SP500_lag = data_q.iloc[222:559, 1].values  # S&P 500 index, lagged (1926:3-2010:3)
DY = np.log(D4) - np.log(SP500_lag)  # log dividend yield
E4 = data_q.iloc[223:560, 3].values  # earnings
EP = np.log(E4) - np.log(SP500)  # log earnings-price ratio
DE = np.log(D4) - np.log(E4)  # log dividend-payout ratio
SVAR = data_q.iloc[223:560, 14].values  # volatility
BM = data_q.iloc[223:560, 4].values  # book-to-market ratio
NTIS = data_q.iloc[223:560, 9].values  # net equity issuing activity
TBL = data_q.iloc[223:560, 5].values  # T-bill rate
LTY = data_q.iloc[223:560, 8].values  # long-term government bond yield
LTR = data_q.iloc[223:560, 12].values  # long-term government bond return
TMS = LTY - TBL  # term spread
AAA = data_q.iloc[223:560, 6].values  # AAA-rated corporate bond yield
BAA = data_q.iloc[223:560, 7].values  # BAA-rated corporate bond yield
DFY = BAA - AAA  # default yield spread
CORPR = data_q.iloc[223:560, 13].values  # long-term corporate bond return
DFR = CORPR - LTR  # default return spread
INFL_lag = data_q.iloc[222:559, 11].values  # inflation, lagged (1926:3-2010:3)
ECON = np.column_stack((DP, DY, EP, DE, SVAR, BM, NTIS, TBL, LTY, LTR, TMS, DFY, DFR, INFL_lag))
ECON_sink = np.column_stack((DP, DY, EP, SVAR, BM, NTIS, TBL, LTY, LTR, DFY, DFR, INFL_lag))

In [ ]:
# Sum-of-the-parts variables

E4_lag = data_q.iloc[222:559, 3].values  # earnings, lagged (1926:3-2010:3)
E_growth = (E4 - E4_lag) / E4_lag # earnings growth
DP_SOP = (1 / 4) * D4 / SP500 # D/P
r_f = data_q.iloc[223:560, 10].values # risk-free rate

## Estimating full-sample parameters for Campbell-Thompson restrictions

In [ ]:
beta_full = np.zeros(np.shape(ECON)[1])
T = np.shape(equity_premium)[0]
LHS = equity_premium[1:T]

In [ ]:
for i in range(ECON.shape[1]):
    RHS_i = np.hstack((ECON[0:T-1, i].reshape(-1, 1), np.ones((T-1, 1))))
    results_i = ols(LHS, RHS_i)
    beta_full[i] = results_i['beta'][0]
    print(i)
beta_full[4] = 1 # restricting SVAR slope coefficient to be positive

# NB: following Campbell and Thompson (2008) by using full-sample
# estimates as theoretical 'priors' (although we make sure that
# SVAR slope coefficient is positive)

## Preliminaries

In [ ]:
Y = equity_premium
T = np.shape(Y)[0]
N = np.shape(ECON)[1]
R = (1946 - 1926) * 4 + 1 # in-sample period, 1926:4-1946:4
P_0 = (1956 - 1946) * 4 # holdout out-of-sample period, 1947:1-1956:4
P = T - (R + P_0) # forecast evaluation period, 1957:1-2010:4
theta = 0.75 # discound factor for DMSFE pooled forecast
r = 1 # number of principal components
MA_SOP = 20 * 4 # window size for SOP forecast

## Computing forecasts, 1947:1-2010:4 (includes holdout OOS period)

In [ ]:
FC_HA = np.zeros((P_0 + P, 1))
FC_ECON = np.zeros((P_0 + P, N))
beta_ECON = np.zeros((P_0 + P, N, 2))
FC_ECON_CT = np.zeros((P_0 + P, N))
omega_DMSFE = np.zeros((P_0 + P, N))
FC_OTHER = np.zeros((P_0 + P, 6))
FC_OTHER_CT = np.zeros((P_0 + P, 6))

In [ ]:
for t in range(P_0 + P):

    FC_HA[t] = np.mean(Y[0:R+t])
    
    # Individual predictive regression forecasts

    X_t = ECON[0:R+t-1,:]
    Y_t = Y[1:R+t]
    for i in range(N):
        results_t_i = ols(Y_t, np.hstack((X_t[:,i:i+1], np.ones((R+t-1,1)))))
        FC_ECON[t,i] = np.dot(np.array([[ECON[R+(t-1),i], 1]]), results_t_i['beta']) # scaler output
        beta_ECON[t,i,0] = results_t_i['beta'][0]
        beta_ECON[t,i,1] = results_t_i['bstd'][0]
        if beta_full[i] > 0:
            if results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        elif beta_full[i] < 0:
            if results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        if FC_ECON_CT[t,i] < 0:
            FC_ECON_CT[t,i] = 0
    if t >= P_0:

        # Kitchen sink forecast

        X_t_sink = ECON_sink[0:R+t-1,:]
        results_t_sink = ols(Y_t, np.hstack((X_t_sink, np.ones((R+t-1,1)))))
        FC_OTHER[t,0] = np.dot(np.append(ECON_sink[R+(t-1),:], 1), results_t_sink['beta'])
        if FC_OTHER[t,0] < 0:
            FC_OTHER_CT[t,0] = 0
        else:
            FC_OTHER_CT[t,0] = FC_OTHER[t,0]
        
        # SIC forecast
        
        j_max = 3  # consider models with up to 3 predictors to save time
        SIC_t = np.empty((0, 2))
        for j in range(1, j_max + 1):
            select_j = list(itertools.combinations(range(X_t_sink.shape[1]), j))
            for k in range(len(select_j)):
                X_t_j_k = np.hstack((X_t_sink[0:R+t-1,select_j[k]], np.ones((R+t-1,1))))
                results_t_j_k = ols(Y_t, X_t_j_k)
                SIC_t_j_k = np.log(np.dot(results_t_j_k['resid'].T, results_t_j_k['resid']) / np.shape(Y_t)[0]) + \
                    np.log(np.shape(Y_t)[0]) * np.shape(X_t_j_k)[1] / np.shape(Y_t)[0]
                FC_t_j_k = np.hstack((ECON_sink[R+(t-1),select_j[k]], 1)).dot(results_t_j_k['beta'])
                SIC_t = np.vstack((SIC_t, [SIC_t_j_k, FC_t_j_k]))
        SIC_t_min_index = np.argmin(SIC_t[:,0])
        SIC_t_min = SIC_t[SIC_t_min_index,0]
        FC_OTHER[t,1] = SIC_t[SIC_t_min_index,1];
        if FC_OTHER[t,1] < 0:
            FC_OTHER_CT[t,1] = 0
        else:
            FC_OTHER_CT[t,1] = FC_OTHER[t,1]
       
        # Pooled forecast: simple average

        FC_OTHER[t,2] = np.mean(FC_ECON[t,:])
        if FC_OTHER[t,2] < 0:
            FC_OTHER_CT[t,2] = 0
        else:
            FC_OTHER_CT[t,2] = FC_OTHER[t,2]
        # Pooled forecast: DMSFE

        powers_t = np.arange(t-1, -1, -1).reshape(-1, 1)
        m = np.sum((np.kron(np.ones((1, N)), (theta*np.ones((t, 1))))**powers_t) * 
                   (((np.kron(Y[R:R+t], np.ones((1, N))).reshape(t, N) - FC_ECON[0:t, :])**2)), axis=0)
        omega = m**(-1) / np.sum(m**(-1))
        FC_OTHER[t, 3] = FC_ECON[t, :].dot(omega)
        if FC_OTHER[t,3] < 0:
            FC_OTHER_CT[t,3] = 0
        else:
            FC_OTHER_CT[t,3] = FC_OTHER[t,3]
        
        # Diffusion index forecast

        X_t_DI_standardize = zscore(ECON[0:R+t, :])
        pca = PCA()
        pca.fit(X_t_DI_standardize)
        Lambda_t = pca.components_.T
        F_t = pca.transform(X_t_DI_standardize)
        results_t_DI = ols(Y_t, np.hstack((F_t[0:-1, 0:r], np.ones((R+t-1,1)))))
        FC_OTHER[t,4] = np.dot(np.append(F_t[-1,0:r], 1), results_t_DI['beta'])
        if FC_OTHER[t,4] < 0:
            FC_OTHER_CT[t,4] = 0
        else:
            FC_OTHER_CT[t,4] = FC_OTHER[t,4]

        # Sum-of-the-parts forecast

        FC_OTHER[t, 5] = np.mean(E_growth[R+(t-1)-MA_SOP+1:R+t]) + DP_SOP[R+(t-1)] - r_f[R+(t-1)]

        if FC_OTHER[t,5] < 0:
            FC_OTHER_CT[t,5] = 0
        else:
            FC_OTHER_CT[t,5] = FC_OTHER[t,5]
        
    print(t, FC_HA[t], FC_OTHER[t, :])

## Forecast evaluation based on asselt allocation exercise, 1957:1-2010:4

In [ ]:
actual = Y[R+P_0:]
FC_HA = FC_HA[P_0:]
FC_ECON = FC_ECON[P_0:,:]
FC_ECON_CT = FC_ECON_CT[P_0:,:]
FC_OTHER = FC_OTHER[P_0:,:]
FC_OTHER_CT = FC_OTHER_CT[P_0:,:]
REC = data_q.iloc[344:560, 18].values # recession dummies, 1957:1-2010:4
EXP = 1 - REC
index_EXP = np.where(EXP)[0]
index_REC = np.where(REC)[0]
gamma_MV = 5 # coefficient of relative risk aversion
window_VOL = 4 * 5 # window size for estimating volatility
FC_VOL = np.zeros((P,1))

In [ ]:
for t in range(P):
    FC_VOL[t,:] = np.mean(Y[R+P_0+(t-1)-window_VOL+1: R+P_0+t]**2) \
        - (np.mean(Y[R+P_0+(t-1)-window_VOL+1: R+P_0+t]))**2

In [ ]:
r_f_lag_P = r_f_lag[R+P_0: R+P_0+P]
U_HA = np.zeros((3,1))
U_HA[0,:], w_HA = Perform_asset_allocation(actual, r_f_lag_P, FC_HA,FC_VOL, gamma_MV)
U_HA[1,:], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_HA[index_EXP], 
                                          FC_VOL[index_EXP], gamma_MV)
U_HA[2,:], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_HA[index_REC], 
                                          FC_VOL[index_REC], gamma_MV)
U_ECON = np.zeros((FC_ECON.shape[1] ,3))
U_ECON_CT = np.zeros((FC_ECON.shape[1] ,3))
w_ECON = np.zeros((P, FC_ECON.shape[1]))
w_ECON_CT = np.zeros((P, FC_ECON_CT.shape[1]))

In [ ]:
for i in range(U_ECON.shape[0]):

    # Overall

    U_ECON[i,0], w_ECON[:,i] = Perform_asset_allocation(actual, r_f_lag_P,FC_ECON[:,i], FC_VOL, gamma_MV)
    U_ECON_CT[i,0], w_ECON_CT[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_ECON_CT[:,i], FC_VOL, gamma_MV)

    # Expansion

    U_ECON[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_ECON[index_EXP,i], 
                                                FC_VOL[index_EXP], gamma_MV)
    U_ECON_CT[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_ECON_CT[index_EXP,i], 
                                                   FC_VOL[index_EXP], gamma_MV)

    # Recession

    U_ECON[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_ECON[index_REC,i], 
                                                FC_VOL[index_REC], gamma_MV)
    U_ECON_CT[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_ECON_CT[index_REC,i], 
                                                   FC_VOL[index_REC], gamma_MV)

In [ ]:
delta_ECON = 400 * (U_ECON - np.kron(U_HA.T, np.ones((FC_ECON.shape[1],1))))
delta_ECON_CT = 400 * (U_ECON_CT - np.kron(U_HA.T, np.ones((FC_ECON_CT.shape[1],1))))
U_OTHER = np.zeros((FC_OTHER.shape[1], 3))
U_OTHER_CT = np.zeros((FC_OTHER_CT.shape[1], 3))
w_OTHER = np.zeros((P, FC_OTHER.shape[1]))
w_OTHER_CT = np.zeros((P, FC_OTHER_CT.shape[1]))

In [ ]:
for i in range(U_OTHER.shape[0]):

    # Overall

    U_OTHER[i,0], w_OTHER[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_OTHER[:,i], FC_VOL,gamma_MV)
    U_OTHER_CT[i,0], w_OTHER_CT[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_OTHER_CT[:,i], FC_VOL,gamma_MV)

    # Expansion

    U_OTHER[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_OTHER[index_EXP,i],
                                                 FC_VOL[index_EXP], gamma_MV)
    U_OTHER_CT[i,1], xxx = Perform_asset_allocation(actual[index_EXP], r_f_lag_P[index_EXP], FC_OTHER_CT[index_EXP,i],
                                                   FC_VOL[index_EXP], gamma_MV)

    # Recession

    U_OTHER[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_OTHER[index_REC,i],
                                                 FC_VOL[index_REC], gamma_MV)
    U_OTHER_CT[i,2], xxx = Perform_asset_allocation(actual[index_REC], r_f_lag_P[index_REC], FC_OTHER_CT[index_REC,i],
                                                    FC_VOL[index_REC], gamma_MV)

In [ ]:
delta_OTHER = 400 * (U_OTHER - np.kron(U_HA.T, np.ones((FC_OTHER.shape[1], 1))))
delta_OTHER_CT = 400 * (U_OTHER_CT - np.kron(U_HA.T, np.ones((FC_OTHER_CT.shape[1], 1))))

## Writing results to spreadsheet

In [ ]:
wb = xw.Book('Returns_handbook_results.xlsx')
sht = wb.sheets['Quarterly']
sht.range('D5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON[:,0:1])
sht.range('D20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER[:,0:1])
sht.range('P5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON_CT[:,0:1])
sht.range('P20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER_CT[:,0:1])
sht.range('H5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON[:,1:2])
sht.range('H20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER[:,1:2])
sht.range('T5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON_CT[:,1:2])
sht.range('T20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER_CT[:,1:2])
sht.range('L5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON[:,2:3])
sht.range('L20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER[:,2:3])
sht.range('X5').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON_CT[:,2:3])
sht.range('X20').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER_CT[:,2:3])
   

# Forecast Annual Log

## Loading data and defining variables, 1926-2010

In [ ]:
data_a = pd.read_excel('Returns_handbook_data.xls', sheet_name='Annual')

In [ ]:
data_a

In [ ]:
# Equity premium

market_return = data_a.iloc[55:140, 15].values  # S&P 500 VW return
r_f_lag = data_a.iloc[54:139, 10].values # risk-free rate, lagged (1925-2009)
equity_premium = np.log(1 + market_return) - np.log(1 + r_f_lag) # log excess return

In [ ]:
# Predictors

D = data_a.iloc[55:140, 2].values
SP500 = data_a.iloc[55:140, 1].values  # S&P 500 index
DP = np.log(D) - np.log(SP500)  # log dividend-price ratio
SP500_lag = data_a.iloc[54:139, 1].values  # S&P 500 index, lagged (1925-2009)
DY = np.log(D) - np.log(SP500_lag)  # log dividend yield
E = data_a.iloc[55:140, 3].values  # earnings
EP = np.log(E) - np.log(SP500)  # log earnings-price ratio
DE = np.log(D) - np.log(E)  # log dividend-payout ratio
SVAR = data_a.iloc[55:140, 14].values  # volatility
BM = data_a.iloc[55:140, 4].values  # book-to-market ratio
NTIS = data_a.iloc[55:140, 9].values  # net equity issuing activity
TBL = data_a.iloc[55:140, 5].values  # T-bill rate
LTY = data_a.iloc[55:140, 8].values  # long-term government bond yield
LTR = data_a.iloc[55:140, 12].values  # long-term government bond return
TMS = LTY - TBL  # term spread
AAA = data_a.iloc[55:140, 6].values  # AAA-rated corporate bond yield
BAA = data_a.iloc[55:140, 7].values  # BAA-rated corporate bond yield
DFY = BAA - AAA  # default yield spread
CORPR = data_a.iloc[55:140, 13].values  # long-term corporate bond return
DFR = CORPR - LTR  # default return spread
INFL_lag = data_a.iloc[54:139, 11].values  # inflation, lagged (1925-2009)
ECON = np.column_stack((DP, DY, EP, DE, SVAR, BM, NTIS, TBL, LTY, LTR, TMS, DFY, DFR, INFL_lag))
ECON_sink = np.column_stack((DP, DY, EP, SVAR, BM, NTIS, TBL, LTY, LTR, DFY, DFR, INFL_lag))

In [ ]:
# Sum-of-the-parts variables

E_lag = data_a.iloc[54:139, 3].values  # earnings, lagged (1925-2009)
E_growth = np.log(E) - np.log(E_lag) # earnings growth
DP_SOP = np.log(1 + np.divide(D, SP500)) # log (1+D/P)
r_f = data_a.iloc[55:140, 10].values # risk-free rate
r_f = np.log(1 + r_f) # log risk-free rate

## Estimating full-sample parameters for Campbell-Thompson restrictions

In [ ]:
beta_full = np.zeros(np.shape(ECON)[1])
T = np.shape(equity_premium)[0]
LHS = equity_premium[1:T]

In [ ]:
for i in range(ECON.shape[1]):
    RHS_i = np.hstack((ECON[0:T-1, i].reshape(-1, 1), np.ones((T-1, 1))))
    results_i = ols(LHS, RHS_i)
    beta_full[i] = results_i['beta'][0]
    print(i)
beta_full[4] = 1 # restricting SVAR slope coefficient to be positive

# NB: following Campbell and Thompson (2008) by using full-sample
# estimates as theoretical 'priors' (although we make sure that
# SVAR slope coefficient is positive)

## Preliminaries

In [ ]:
Y = equity_premium
T = np.shape(Y)[0]
N = np.shape(ECON)[1]
R = (1946 - 1926) + 1 # in-sample period, 1926-1946
P_0 = (1956 - 1946) # holdout out-of-sample period, 1947-1956
P = T - (R + P_0) # forecast evaluation period, 1957-2010
theta = 0.75 # discound factor for DMSFE pooled forecast
r = 1 # number of principal components
MA_SOP = 20 # window size for SOP forecast

## Computing forecasts, 1947-2010 (includes holdout OOS period)

In [ ]:
FC_HA = np.zeros((P_0 + P, 1))
FC_ECON = np.zeros((P_0 + P, N))
beta_ECON = np.zeros((P_0 + P, N, 2))
FC_ECON_CT = np.zeros((P_0 + P, N))
omega_DMSFE = np.zeros((P_0 + P, N))
FC_OTHER = np.zeros((P_0 + P, 6))
FC_OTHER_CT = np.zeros((P_0 + P, 6))

In [ ]:
for t in range(P_0 + P):

    FC_HA[t] = np.mean(Y[0:R+t])
    
    # Individual predictive regression forecasts

    X_t = ECON[0:R+t-1,:]
    Y_t = Y[1:R+t]
    for i in range(N):
        results_t_i = ols(Y_t, np.hstack((X_t[:,i:i+1], np.ones((R+t-1,1)))))
        FC_ECON[t,i] = np.dot(np.array([[ECON[R+(t-1),i], 1]]), results_t_i['beta']) # scaler output
        beta_ECON[t,i,0] = results_t_i['beta'][0]
        beta_ECON[t,i,1] = results_t_i['bstd'][0]
        if beta_full[i] > 0:
            if results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        elif beta_full[i] < 0:
            if results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        if FC_ECON_CT[t,i] < 0:
            FC_ECON_CT[t,i] = 0
    if t >= P_0:

        # Kitchen sink forecast

        X_t_sink = ECON_sink[0:R+t-1,:]
        results_t_sink = ols(Y_t, np.hstack((X_t_sink, np.ones((R+t-1,1)))))
        FC_OTHER[t,0] = np.dot(np.append(ECON_sink[R+(t-1),:], 1), results_t_sink['beta'])
        if FC_OTHER[t,0] < 0:
            FC_OTHER_CT[t,0] = 0
        else:
            FC_OTHER_CT[t,0] = FC_OTHER[t,0]
        
        # SIC forecast
        
        j_max = 3  # consider models with up to 3 predictors to save time
        SIC_t = np.empty((0, 2))
        for j in range(1, j_max + 1):
            select_j = list(itertools.combinations(range(X_t_sink.shape[1]), j))
            for k in range(len(select_j)):
                X_t_j_k = np.hstack((X_t_sink[0:R+t-1,select_j[k]], np.ones((R+t-1,1))))
                results_t_j_k = ols(Y_t, X_t_j_k)
                SIC_t_j_k = np.log(np.dot(results_t_j_k['resid'].T, results_t_j_k['resid']) / np.shape(Y_t)[0]) + \
                    np.log(np.shape(Y_t)[0]) * np.shape(X_t_j_k)[1] / np.shape(Y_t)[0]
                FC_t_j_k = np.hstack((ECON_sink[R+(t-1),select_j[k]], 1)).dot(results_t_j_k['beta'])
                SIC_t = np.vstack((SIC_t, [SIC_t_j_k, FC_t_j_k]))
        SIC_t_min_index = np.argmin(SIC_t[:,0])
        SIC_t_min = SIC_t[SIC_t_min_index,0]
        FC_OTHER[t,1] = SIC_t[SIC_t_min_index,1];
        if FC_OTHER[t,1] < 0:
            FC_OTHER_CT[t,1] = 0
        else:
            FC_OTHER_CT[t,1] = FC_OTHER[t,1]

        # Pooled forecast: simple average

        FC_OTHER[t,2] = np.mean(FC_ECON[t,:])
        if FC_OTHER[t,2] < 0:
            FC_OTHER_CT[t,2] = 0
        else:
            FC_OTHER_CT[t,2] = FC_OTHER[t,2]
            
        # Pooled forecast: DMSFE

        powers_t = np.arange(t-1, -1, -1).reshape(-1, 1)
        m = np.sum((np.kron(np.ones((1, N)), (theta*np.ones((t, 1))))**powers_t) * 
                   (((np.kron(Y[R:R+t], np.ones((1, N))).reshape(t, N) - FC_ECON[0:t, :])**2)), axis=0)
        omega = m**(-1) / np.sum(m**(-1))
        omega_DMSFE[t, :] = omega
        FC_OTHER[t, 3] = FC_ECON[t, :].dot(omega)
        if FC_OTHER[t,3] < 0:
            FC_OTHER_CT[t,3] = 0
        else:
            FC_OTHER_CT[t,3] = FC_OTHER[t,3]

        # Diffusion index forecast

        X_t_DI_standardize = zscore(ECON[0:R+t, :])
        pca = PCA()
        pca.fit(X_t_DI_standardize)
        Lambda_t = pca.components_.T
        F_t = pca.transform(X_t_DI_standardize)
        results_t_DI = ols(Y_t, np.hstack((F_t[0:-1, 0:r], np.ones((R+t-1,1)))))
        FC_OTHER[t,4] = np.dot(np.append(F_t[-1,0:r], 1), results_t_DI['beta'])
        if FC_OTHER[t,4] < 0:
            FC_OTHER_CT[t,4] = 0
        else:
            FC_OTHER_CT[t,4] = FC_OTHER[t,4]

        # Sum-of-the-parts forecast

        FC_OTHER[t, 5] = np.mean(E_growth[R+(t-1)-MA_SOP+1:R+t]) + DP_SOP[R+(t-1)] - r_f[R+(t-1)]

        if FC_OTHER[t,5] < 0:
            FC_OTHER_CT[t,5] = 0
        else:
            FC_OTHER_CT[t,5] = FC_OTHER[t,5]
        
    print(t, FC_HA[t], FC_OTHER[t, :])

## Forecast evaluation based on MSFE, 1957-2010

In [ ]:
beta_ECON = beta_ECON[P_0:,:,:]
actual = Y[R+P_0:]
FC_HA = FC_HA[P_0:]
FC_ECON = FC_ECON[P_0:,:]
FC_ECON_CT = FC_ECON_CT[P_0:,:]
FC_OTHER = FC_OTHER[P_0:,:]
FC_OTHER_CT = FC_OTHER_CT[P_0:,:]
omega_DMSFE = omega_DMSFE[P_0:,:]
e_HA = actual.reshape(actual.shape[0],1) - FC_HA
e_ECON = np.kron(np.ones((FC_ECON.shape[1], 1)), actual).T - FC_ECON
e_ECON_CT = np.kron(np.ones((FC_ECON_CT.shape[1], 1)), actual).T - FC_ECON_CT
CSFE_HA = np.cumsum(e_HA**2)
CSFE_ECON = np.cumsum(e_ECON**2, axis=0)
CSFE_ECON_CT = np.cumsum(e_ECON_CT**2, axis=0)
DCSFE_ECON = np.kron(np.ones((FC_ECON.shape[1], 1)), CSFE_HA).T - CSFE_ECON
DCSFE_ECON_CT = np.kron(np.ones((FC_ECON_CT.shape[1], 1)), CSFE_HA).T - CSFE_ECON_CT
R2OS_ECON = np.zeros((FC_ECON.shape[1], 2))
R2OS_ECON_CT = np.zeros((FC_ECON_CT.shape[1], 2))

In [ ]:
for i in range(R2OS_ECON.shape[0]):
    R2OS_ECON[i, 0] = 100*(1 - np.sum(e_ECON[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_ECON[:, i:i+1]**2 - (FC_HA - FC_ECON[:, i:i+1])**2)
    results_i = nwest(f_i.flatten() , np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_ECON_CT[i, 0] = 100*(1 - np.sum(e_ECON_CT[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_ECON_CT[:, i:i+1]**2 - (FC_HA - FC_ECON_CT[:, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_ECON_CT[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)

In [ ]:
e_OTHER = np.kron(np.ones((FC_OTHER.shape[1], 1)), actual).T - FC_OTHER
e_OTHER_CT = np.kron(np.ones((FC_OTHER_CT.shape[1], 1)), actual).T - FC_OTHER_CT
CSFE_OTHER = np.cumsum(e_OTHER**2, axis=0)
CSFE_OTHER_CT = np.cumsum(e_OTHER_CT**2, axis=0)
DCSFE_OTHER = np.kron(np.ones((FC_OTHER.shape[1], 1)), CSFE_HA).T - CSFE_OTHER
DCSFE_OTHER_CT = np.kron(np.ones((FC_OTHER_CT.shape[1], 1)), CSFE_HA).T - CSFE_OTHER_CT
R2OS_OTHER = np.zeros((FC_OTHER.shape[1], 2))
R2OS_OTHER_CT = np.zeros((FC_OTHER_CT.shape[1], 2))

In [ ]:
for i in range(R2OS_OTHER.shape[0]):
    R2OS_OTHER[i, 0] = 100 * (1 - np.sum(e_OTHER[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_OTHER[:, i:i+1]**2 - (FC_HA - FC_OTHER[:, i:i+1])**2)
    results_i = nwest(f_i.flatten() , np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)
    R2OS_OTHER_CT[i, 0] = 100 * (1 - np.sum(e_OTHER_CT[:, i]**2) / np.sum(e_HA**2))
    f_i = e_HA**2 - (e_OTHER_CT[:, i:i+1]**2 - (FC_HA - FC_OTHER_CT[:, i:i+1])**2)
    results_i = nwest(f_i.flatten(), np.ones((f_i.shape[0], 1)), 0)
    R2OS_OTHER_CT[i, 1] = 1 - norm.cdf(results_i['tstat'], 0, 1)

## Writing results to spreadsheet

In [ ]:
wb = xw.Book('Returns_handbook_results.xlsx')
sht = wb.sheets['Annual']
sht.range('B4').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON)
sht.range('F4').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_ECON_CT)
sht.range('B19').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER)
sht.range('F19').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(R2OS_OTHER_CT)
                                                                                                   

# Forecast Annual

## Loading data and defining variables, 1926-2010

In [ ]:
data_a = pd.read_excel('Returns_handbook_data.xls', sheet_name='Annual')

In [ ]:
data_a

In [ ]:
# Equity premium

market_return = data_a.iloc[55:140, 15].values  # S&P 500 VW return
r_f_lag = data_a.iloc[54:139, 10].values # risk-free rate, lagged (1925-2009)
equity_premium = market_return - r_f_lag # excess return

In [ ]:
# Predictors

D = data_a.iloc[55:140, 2].values
SP500 = data_a.iloc[55:140, 1].values  # S&P 500 index
DP = np.log(D) - np.log(SP500)  # log dividend-price ratio
SP500_lag = data_a.iloc[54:139, 1].values  # S&P 500 index, lagged (1925-2009)
DY = np.log(D) - np.log(SP500_lag)  # log dividend yield
E = data_a.iloc[55:140, 3].values  # earnings
EP = np.log(E) - np.log(SP500)  # log earnings-price ratio
DE = np.log(D) - np.log(E)  # log dividend-payout ratio
SVAR = data_a.iloc[55:140, 14].values  # volatility
BM = data_a.iloc[55:140, 4].values  # book-to-market ratio
NTIS = data_a.iloc[55:140, 9].values  # net equity issuing activity
TBL = data_a.iloc[55:140, 5].values  # T-bill rate
LTY = data_a.iloc[55:140, 8].values  # long-term government bond yield
LTR = data_a.iloc[55:140, 12].values  # long-term government bond return
TMS = LTY - TBL  # term spread
AAA = data_a.iloc[55:140, 6].values  # AAA-rated corporate bond yield
BAA = data_a.iloc[55:140, 7].values  # BAA-rated corporate bond yield
DFY = BAA - AAA  # default yield spread
CORPR = data_a.iloc[55:140, 13].values  # long-term corporate bond return
DFR = CORPR - LTR  # default return spread
INFL_lag = data_a.iloc[54:139, 11].values  # inflation, lagged (1925-2009)
ECON = np.column_stack((DP, DY, EP, DE, SVAR, BM, NTIS, TBL, LTY, LTR, TMS, DFY, DFR, INFL_lag))
ECON_sink = np.column_stack((DP, DY, EP, SVAR, BM, NTIS, TBL, LTY, LTR, DFY, DFR, INFL_lag))

In [ ]:
# Sum-of-the-parts variables

E_lag = data_a.iloc[54:139, 3].values  # earnings, lagged (1925-2009)
E_growth = (E - E_lag) / E_lag # earnings growth
DP_SOP = D / SP500 # D/P
r_f = data_a.iloc[55:140, 10].values # risk-free rate

## Estimating full-sample parameters for Campbell-Thompson restrictions

In [ ]:
beta_full = np.zeros(np.shape(ECON)[1])
T = np.shape(equity_premium)[0]
LHS = equity_premium[1:T]

In [ ]:
for i in range(ECON.shape[1]):
    RHS_i = np.hstack((ECON[0:T-1, i].reshape(-1, 1), np.ones((T-1, 1))))
    results_i = ols(LHS, RHS_i)
    beta_full[i] = results_i['beta'][0]
    print(i)
beta_full[4] = 1 # restricting SVAR slope coefficient to be positive

# NB: following Campbell and Thompson (2008) by using full-sample
# estimates as theoretical 'priors' (although we make sure that
# SVAR slope coefficient is positive)

## Preliminaries

In [ ]:
Y = equity_premium
T = np.shape(Y)[0]
N = np.shape(ECON)[1]
R = (1946 - 1926) + 1 # in-sample period, 1926-1946
P_0 = (1956 - 1946) # holdout out-of-sample period, 1947-1956
P = T - (R + P_0) # forecast evaluation period, 1957-2010
theta = 0.75 # discound factor for DMSFE pooled forecast
r = 1 # number of principal components
MA_SOP = 20 # window size for SOP forecast

## Computing forecasts, 1947-2010 (includes holdout OOS period)

In [ ]:
FC_HA = np.zeros((P_0 + P, 1))
FC_ECON = np.zeros((P_0 + P, N))
beta_ECON = np.zeros((P_0 + P, N, 2))
FC_ECON_CT = np.zeros((P_0 + P, N))
FC_OTHER = np.zeros((P_0 + P, 6))
FC_OTHER_CT = np.zeros((P_0 + P, 6))

In [ ]:
for t in range(P_0 + P):

    FC_HA[t] = np.mean(Y[0:R+t])
    
    # Individual predictive regression forecasts

    X_t = ECON[0:R+t-1,:]
    Y_t = Y[1:R+t]
    for i in range(N):
        results_t_i = ols(Y_t, np.hstack((X_t[:,i:i+1], np.ones((R+t-1,1)))))
        FC_ECON[t,i] = np.dot(np.array([[ECON[R+(t-1),i], 1]]), results_t_i['beta']) # scaler output
        beta_ECON[t,i,0] = results_t_i['beta'][0]
        beta_ECON[t,i,1] = results_t_i['bstd'][0]
        if beta_full[i] > 0:
            if results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        elif beta_full[i] < 0:
            if results_t_i['beta'][0] < 0:
                FC_ECON_CT[t,i] = FC_ECON[t,i]
            elif results_t_i['beta'][0] > 0:
                FC_ECON_CT[t,i] = FC_HA[t]
        if FC_ECON_CT[t,i] < 0:
            FC_ECON_CT[t,i] = 0
    if t >= P_0:

        # Kitchen sink forecast

        X_t_sink = ECON_sink[0:R+t-1,:]
        results_t_sink = ols(Y_t, np.hstack((X_t_sink, np.ones((R+t-1,1)))))
        FC_OTHER[t,0] = np.dot(np.append(ECON_sink[R+(t-1),:], 1), results_t_sink['beta'])
        if FC_OTHER[t,0] < 0:
            FC_OTHER_CT[t,0] = 0
        else:
            FC_OTHER_CT[t,0] = FC_OTHER[t,0]
        
        # SIC forecast
        
        j_max = 3  # consider models with up to 3 predictors to save time
        SIC_t = np.empty((0, 2))
        for j in range(1, j_max + 1):
            select_j = list(itertools.combinations(range(X_t_sink.shape[1]), j))
            for k in range(len(select_j)):
                X_t_j_k = np.hstack((X_t_sink[0:R+t-1,select_j[k]], np.ones((R+t-1,1))))
                results_t_j_k = ols(Y_t, X_t_j_k)
                SIC_t_j_k = np.log(np.dot(results_t_j_k['resid'].T, results_t_j_k['resid']) / np.shape(Y_t)[0]) + \
                    np.log(np.shape(Y_t)[0]) * np.shape(X_t_j_k)[1] / np.shape(Y_t)[0]
                FC_t_j_k = np.hstack((ECON_sink[R+(t-1),select_j[k]], 1)).dot(results_t_j_k['beta'])
                SIC_t = np.vstack((SIC_t, [SIC_t_j_k, FC_t_j_k]))
        SIC_t_min_index = np.argmin(SIC_t[:,0])
        SIC_t_min = SIC_t[SIC_t_min_index,0]
        FC_OTHER[t,1] = SIC_t[SIC_t_min_index,1];
        if FC_OTHER[t,1] < 0:
            FC_OTHER_CT[t,1] = 0
        else:
            FC_OTHER_CT[t,1] = FC_OTHER[t,1]

        # Pooled forecast: simple average

        FC_OTHER[t,2] = np.mean(FC_ECON[t,:])
        if FC_OTHER[t,2] < 0:
            FC_OTHER_CT[t,2] = 0
        else:
            FC_OTHER_CT[t,2] = FC_OTHER[t,2]
            

        # Pooled forecast: DMSFE

        powers_t = np.arange(t-1, -1, -1).reshape(-1, 1)
        m = np.sum((np.kron(np.ones((1, N)), (theta*np.ones((t, 1))))**powers_t) * 
                   (((np.kron(Y[R:R+t], np.ones((1, N))).reshape(t, N) - FC_ECON[0:t, :])**2)), axis=0)
        omega = m**(-1) / np.sum(m**(-1))
        FC_OTHER[t, 3] = FC_ECON[t, :].dot(omega)
        if FC_OTHER[t,3] < 0:
            FC_OTHER_CT[t,3] = 0
        else:
            FC_OTHER_CT[t,3] = FC_OTHER[t,3]

        # Diffusion index forecast

        X_t_DI_standardize = zscore(ECON[0:R+t, :])
        pca = PCA()
        pca.fit(X_t_DI_standardize)
        Lambda_t = pca.components_.T
        F_t = pca.transform(X_t_DI_standardize)
        results_t_DI = ols(Y_t, np.hstack((F_t[0:-1, 0:r], np.ones((R+t-1,1)))))
        FC_OTHER[t,4] = np.dot(np.append(F_t[-1,0:r], 1), results_t_DI['beta'])
        if FC_OTHER[t,4] < 0:
            FC_OTHER_CT[t,4] = 0
        else:
            FC_OTHER_CT[t,4] = FC_OTHER[t,4]

        # Sum-of-the-parts forecast

        FC_OTHER[t, 5] = np.mean(E_growth[R+(t-1)-MA_SOP+1:R+t]) + DP_SOP[R+(t-1)] - r_f[R+(t-1)]

        if FC_OTHER[t,5] < 0:
            FC_OTHER_CT[t,5] = 0
        else:
            FC_OTHER_CT[t,5] = FC_OTHER[t,5]
        
    print(t, FC_HA[t], FC_OTHER[t, :])

## Forecast evaluation based on asselt allocation exercise, 1957-2010

In [ ]:
actual = Y[R+P_0:]
FC_HA = FC_HA[P_0:]
FC_ECON = FC_ECON[P_0:,:]
FC_ECON_CT = FC_ECON_CT[P_0:,:]
FC_OTHER = FC_OTHER[P_0:,:]
FC_OTHER_CT = FC_OTHER_CT[P_0:,:]
gamma_MV = 5 # coefficient of relative risk aversion
window_VOL = 5 # window size for estimating volatility
FC_VOL = np.zeros((P,1))

In [ ]:
for t in range(P):
    FC_VOL[t,:] = np.mean(Y[R+P_0+(t-1)-window_VOL+1: R+P_0+t]**2) \
        - (np.mean(Y[R+P_0+(t-1)-window_VOL+1: R+P_0+t]))**2

In [ ]:
r_f_lag_P = r_f_lag[R+P_0: R+P_0+P]
U_HA, w_HA = Perform_asset_allocation(actual, r_f_lag_P, FC_HA,FC_VOL, gamma_MV)
U_ECON = np.zeros((FC_ECON.shape[1] ,1))
U_ECON_CT = np.zeros((FC_ECON.shape[1] ,1))
w_ECON = np.zeros((P, FC_ECON.shape[1]))
w_ECON_CT = np.zeros((P, FC_ECON_CT.shape[1]))

In [ ]:
for i in range(U_ECON.shape[0]):
    U_ECON[i,0], w_ECON[:,i] = Perform_asset_allocation(actual, r_f_lag_P,FC_ECON[:,i], FC_VOL, gamma_MV)
    U_ECON_CT[i,0], w_ECON_CT[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_ECON_CT[:,i], FC_VOL, gamma_MV)

In [ ]:
delta_ECON = 100 * (U_ECON - np.kron(U_HA.T, np.ones((FC_ECON.shape[1],1))))
delta_ECON_CT = 100 * (U_ECON_CT - np.kron(U_HA.T, np.ones((FC_ECON_CT.shape[1],1))))
U_OTHER = np.zeros((FC_OTHER.shape[1], 1))
U_OTHER_CT = np.zeros((FC_OTHER_CT.shape[1], 1))
w_OTHER = np.zeros((P, FC_OTHER.shape[1]))
w_OTHER_CT = np.zeros((P, FC_OTHER_CT.shape[1]))

In [ ]:
for i in range(U_OTHER.shape[0]):
    U_OTHER[i,0], w_OTHER[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_OTHER[:,i], FC_VOL,gamma_MV)
    U_OTHER_CT[i,0], w_OTHER_CT[:,i] = Perform_asset_allocation(actual, r_f_lag_P, FC_OTHER_CT[:,i], FC_VOL,gamma_MV)

In [ ]:
delta_OTHER = 100 * (U_OTHER - np.kron(U_HA.T, np.ones((FC_OTHER.shape[1], 1))))
delta_OTHER_CT = 100 * (U_OTHER_CT - np.kron(U_HA.T, np.ones((FC_OTHER_CT.shape[1], 1))))

## Writing results to spreadsheet

In [ ]:
wb = xw.Book('Returns_handbook_results.xlsx')
sht = wb.sheets['Annual']
sht.range('D4').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON[:,0:1])
sht.range('D19').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER[:,0:1])
sht.range('H4').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_ECON_CT[:,0:1])
sht.range('H19').options(pd.DataFrame, header=0, index=False, expand='table').value = pd.DataFrame(delta_OTHER_CT[:,0:1])
                                                                                                   